# Les permis de construire délivrés sur le littoral breton (2014-2024)

*Denis Vannier - Le Plan Studio - 2024 - https://leplan.studio/ - https://github.com/Denis-Vannier*

**SOURCES**

- Base SITADEL (Ministère de la Transition écologique). Autorisations de construction de logements et de locaux, délivrées entre janvier 2013 et novembre 2024 : https://www.statistiques.developpement-durable.gouv.fr/liste-des-permis-de-construire-et-autres-autorisations-durbanisme

- Base Sitadel géolocalisée par Koumoul : https://opendata.koumoul.com/datasets/sitadel-locaux et https://opendata.koumoul.com/datasets/sitadel-logements 

- Cadastre géolocalisé par Etalab (Compilation de l'ensemble des millésimes disponibles (2017-2024)) : https://cadastre.data.gouv.fr/datasets/cadastre-etalab

- Admin Express (IGN) https://geoservices.ign.fr/adminexpress 

- Découpage par bandes littorales (voir traitements BANDE_LITTORALE_FRANCE.ipynb )



In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.9f' % x)
import requests
import datetime
import matplotlib.pyplot as plt
import geopandas as gpd
import mapclassify
import os
import numpy as np
import shapely
from shapely.geometry import Point, Polygon

In [9]:
dep=["22","29","35","56","44"]
comFR=gpd.read_file("ADMIN-EXPRESS/1_DONNEES_LIVRAISON_2023-12-00156/ADE_3-2_SHP_LAMB93_FXX-ED2023-12-19/COMMUNE.shp")
comFR=comFR[["INSEE_DEP","INSEE_COM","NOM","SIREN_EPCI","geometry"]]
comFR["INSEE_DEP"]=comFR["INSEE_DEP"].astype("str")
Com_BZH=comFR[comFR["INSEE_DEP"].isin(dep)]

epciFR=gpd.read_file("ADMIN-EXPRESS/1_DONNEES_LIVRAISON_2023-12-00156/ADE_3-2_SHP_LAMB93_FXX-ED2023-12-19/EPCI.shp")
epciFR=epciFR[["CODE_SIREN","NOM"]]
epciFR.rename(columns={"CODE_SIREN":"SIREN_EPCI", "NOM":"NOM_EPCI"}, inplace=True)
epciFR["SIREN_EPCI"]=epciFR["SIREN_EPCI"].astype("str")
Com_BZH["SIREN_EPCI"]=Com_BZH["SIREN_EPCI"].astype("str")
Com_BZH=Com_BZH.merge(epciFR, on="SIREN_EPCI", how="left")
Com_BZH.rename(columns={"NOM":"NOM_COMMUNE", "INSEE_DEP":"DEP"}, inplace=True)
Com_BZH=Com_BZH[["DEP","INSEE_COM","NOM_COMMUNE","SIREN_EPCI", "NOM_EPCI"]]
Com_BZH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1414 entries, 0 to 1413
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   DEP          1414 non-null   object
 1   INSEE_COM    1414 non-null   object
 2   NOM_COMMUNE  1414 non-null   object
 3   SIREN_EPCI   1414 non-null   object
 4   NOM_EPCI     1411 non-null   object
dtypes: object(5)
memory usage: 55.4+ KB


/Users/denisvannier/opt/anaconda3/envs/GEODATA2/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


### Traitement de la version géocodée de SITADEL par Koumoul

Logements Sitadel Koumoul :

In [3]:
Koumoul_loge=pd.read_csv("SITADEL/SITADEL_Koumoul_2014_2024/Koumoul_sitadel-logements.csv")
Koumoul_loge.rename(columns={"COMM":"INSEE_COM", "DEP_CODE":"DEP", "DATE_REELLE_AUTORISATION":"DATE", "NUM_DAU":"NUM_PERMIS", "ETAT_DAU":"ETAT_AVANCEMENT", "CODPOST_DEM":"CP_DEMANDEUR", "LOCALITE_DEM":"COMMUNE_DEMANDEUR", "ADR_NUM_TER":"NUM_VOIE_TERRAIN", "ADR_TYPEVOIE_TER":"TYPE_VOIE-TERRAIN", "ADR_LIBVOIE_TER":"NOM_VOIE_TERRAIN", "ADR_CODPOST_TER":"CP_TERRAIN", "ADR_LOCALITE_TER":"COMMUNE_TERRAIN", "SUPERFICIE_TERRAIN":"SUPERF_TERRAIN", "NATURE_PROJET_DECLAREE":"NATURE_PROJET", "NATURE_PROJET_COMPLETEE":"NATURE_PROJET_DETAIL", "DESTINATION_PRINCIPALE":"DESTINATION", "SURF_HAB_CREEE":"SURF_HABIT_NEW", "SURF_LOC_CREEE":"SURF_LOCAUX_NEW", "NB_LGT_TOT_CREES":"NB_LOGEMENTS_NEW", "RES_PRINCIP_OU_SECOND":"TYPE_HABIT_PRINCIP_SECOND", "APE_DEM":"CODE_ACTIV_DEMANDEUR", "CAT_DEM":"CAT_JURID_DEMANDEUR", "DENOM_DEM":"NOM_DEMANDEUR", "SIREN_DEM":"SIREN_DEMANDEUR", "SIRET_DEM":"SIRET_DEMANDEUR"}, inplace=True)

# On ne retient que les nouvelles constructions : 
Koumoul_loge["NATURE_PROJET"]=Koumoul_loge["NATURE_PROJET"].astype("str")
Koumoul_loge=Koumoul_loge[Koumoul_loge["NATURE_PROJET"]=="1"]

#### Création d'une nouvelle variable permettant de savoir si le permis a été annulé, 
# à partir du code fournir par la variable "ETAT_AVANCEMENT", qu'on supprime ensuite :
Koumoul_loge.loc[Koumoul_loge["ETAT_AVANCEMENT"].astype("str")=="4", "PERMIS_ANNULE"]="oui"
Koumoul_loge.loc[Koumoul_loge["ETAT_AVANCEMENT"].astype("str")!="4", "PERMIS_ANNULE"]="non"
Koumoul_loge.drop(columns="ETAT_AVANCEMENT", inplace=True)


#### Création d'une nouvelle variable permettant de savoir 
# si le demandeur a déclaré qu'il ferait du logement construit sa résidence secondaire :
Koumoul_loge.loc[Koumoul_loge["TYPE_HABIT_PRINCIP_SECOND"].astype("str")=="2", "RESID_SECOND"]="1"
Koumoul_loge["TYPE_HABIT_PRINCIP_SECOND"]=Koumoul_loge["TYPE_HABIT_PRINCIP_SECOND"].fillna("0")

# Création d'une nouvelle variable "ANNEE_MOIS_AUTORISATION" pour préparer une synthèse mensuelle des permis :
Koumoul_loge["DATE"]=Koumoul_loge["DATE"].astype("str")
Koumoul_loge["ANNEE_MOIS_AUTORISATION"]=(Koumoul_loge["DATE"].str[:7])+"-01 01:00:00"
Koumoul_loge["ANNEE_MOIS_AUTORISATION"]= pd.to_datetime(Koumoul_loge["ANNEE_MOIS_AUTORISATION"])

# Modifier les codes insee des communes (qui ont pu changer depuis 2013 suite aux fusions de communes), 
# en utilisant la table de passage (https://www.insee.fr/fr/information/7671867):
Koumoul_loge["INSEE_COM"]=Koumoul_loge["INSEE_COM"].astype("str")
Koumoul_loge.replace({"INSEE_COM": {"22154":"22093", "22173":"22093", "22038":"22206", "22100":"22219", "22078":"22264", "22247":"22264", "22253":"22264", "22123":"22050", "22007":"22055", "22167":"22107", "22290":"22107", "22298":"22158", "22192":"22209", "22357":"22209", "22066":"22046", "22102":"22046", "22191":"22046", "22292":"22046", "22297":"22046", "22303":"22046", "22051":"22084", "22151":"22093", "22058":"22183", "22080":"22203", "22367":"22251", "29219":"29199", "29129":"29227", "29203":"29021", "29149":"29076", "29052":"29003", "29127":"29266", "35112":"35062", "35113":"35004", "35303":"35004", "35341":"35004", "35100":"35163", "35301":"35184", "35053":"35220", "35269":"35282", "35293":"35282", "35348":"35282", "35011":"35292", "35147":"35308", "35344":"35308", "35209":"35069", "35254":"35069", "35048":"35168", "35083":"35191", "35323":"35191", "35267":"35257", "35158":"35060", "35129":"35176", "44160":"44003", "44017":"44180", "44093":"44180", "44191":"44180", "44219":"44180", "44040":"44005", "44059":"44021", "44008":"44029", "44181":"44087", "44004":"44163", "44011":"44213", "44034":"44213", "44147":"44213", "44060":"49160", "56049":"56213", "56059":"56102", "56138":"56165", "56016":"56173", "56183":"56033", "56038":"56061", "56064":"56061", "56142":"56144", "56192":"56144", "56037":"56197", "56187":"56197", "56150":"56251"}}, inplace=True)

# Ajout du statut et de l'Epci de rattachement des communes :
Com_BZH=Com_BZH[["INSEE_COM","NOM_COMMUNE","SIREN_EPCI","NOM_EPCI"]]
Com_BZH["INSEE_COM"]=Com_BZH["INSEE_COM"].astype("str")
Koumoul_loge=Koumoul_loge.merge(Com_BZH, how="left", on="INSEE_COM")


#  Faute d'un code insee pour la commune du demandeur, 
# on doit le recupérer en croisant le nom de commune avec la liste des propre des communes bretonnes, 
# et en profiter pour ajouter le code de l'EPCI et la statut (littoral...)
# On extrait une liste propre des communes :
Detail_Communes=Koumoul_loge[["COMMUNE_TERRAIN", "NOM_COMMUNE", "SIREN_EPCI","INSEE_COM"]]
Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
Detail_Communes.sort_values(by="COMMUNE_TERRAIN", inplace=True)
Detail_Communes.rename(columns={"COMMUNE_TERRAIN":"COMMUNE_DEMANDEUR", "NOM_COMMUNE":"NOM_COMMUNE_ok", "SIREN_EPCI":"EPCI_DEMANDEUR", "INSEE_COM":"DEM_INSEE_COM"}, inplace=True)
# Puis on fusionne avec la base Sitadel :
Koumoul_loge=Koumoul_loge.merge(Detail_Communes, how="left", on="COMMUNE_DEMANDEUR")
Koumoul_loge.replace({"EPCI_DEMANDEUR": {"NR": "0"}}, inplace=True)
Koumoul_loge["EPCI_DEMANDEUR"]=Koumoul_loge["EPCI_DEMANDEUR"].fillna("0").astype("int64")

# Le demandeur du permis est-il domicilié dans la même commune ?
Koumoul_loge.loc[Koumoul_loge["COMMUNE_TERRAIN"]==Koumoul_loge["COMMUNE_DEMANDEUR"],"DEM_MEME_COMMUNE"]="1"
Koumoul_loge["DEM_MEME_COMMUNE"]=Koumoul_loge["DEM_MEME_COMMUNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le même département ?
Koumoul_loge["DEP_DEMANDEUR"]=Koumoul_loge["CP_DEMANDEUR"].str[:2]
Koumoul_loge["DEP_DEMANDEUR"]=Koumoul_loge["DEP_DEMANDEUR"].astype("str")
Koumoul_loge["DEP"]=Koumoul_loge["DEP"].astype("str")
Koumoul_loge["DEP"]=Koumoul_loge["DEP"].str[:2]
Koumoul_loge.loc[Koumoul_loge["DEP"]==Koumoul_loge["DEP_DEMANDEUR"] ,"DEM_MEME_DEP"]="1"
Koumoul_loge["DEM_MEME_DEP"]=Koumoul_loge["DEM_MEME_DEP"].fillna("0")

# Le demandeur est-il du permis est domicilié en Bretagne (à 5dep) ? 
dep_Bzh=("29","22","56","35","44")
Koumoul_loge.loc[Koumoul_loge["DEP_DEMANDEUR"].isin(dep_Bzh),"DEM_BRETAGNE"]="1"
Koumoul_loge["DEM_BRETAGNE"]=Koumoul_loge["DEM_BRETAGNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le quart nord-ouest de la France (Bretagne, Pays de la Loire ou ex-basse-Normandie) ?
Ouest=("44","85","53","72","49","14","50","61","29","22","56","35")
Koumoul_loge.loc[Koumoul_loge["DEP_DEMANDEUR"].isin(Ouest),"DEM_Ouest"]="1"
Koumoul_loge["DEM_Ouest"]=Koumoul_loge["DEM_Ouest"].fillna("0")

# Le demandeur du permis est-il domicilié dans une autre région (autre que Bretagne, Pays de la Loire ou Normandie) ?
Koumoul_loge.loc[Koumoul_loge["DEM_Ouest"]=="0","DEM_LOINTAINS"]="1"
Koumoul_loge["DEM_LOINTAINS"]=Koumoul_loge["DEM_LOINTAINS"].fillna("0")

# Le demandeur est-il du permis est domicilié dans le même EPCI ?
Koumoul_loge["SIREN_EPCI"]=Koumoul_loge["SIREN_EPCI"].astype("str")
Koumoul_loge["EPCI_DEMANDEUR"]=Koumoul_loge["EPCI_DEMANDEUR"].astype("str")
Koumoul_loge.loc[Koumoul_loge["SIREN_EPCI"]==Koumoul_loge["EPCI_DEMANDEUR"],"DEM_MEME_EPCI"]="1"
Koumoul_loge["DEM_MEME_EPCI"]=Koumoul_loge["DEM_MEME_EPCI"].fillna("0")

Koumoul_loge.rename(columns={"NOM_COMMUNE_ok":"NOM_COMMUNE_DEMANDEUR"}, inplace=True)
Koumoul_loge.drop(columns={"COMMUNE_DEMANDEUR","COMMUNE_TERRAIN"}, inplace=True)

Koumoul_loge["Type_AU"]="Logements"

Koumoul_loge=Koumoul_loge[['Type_AU','DEP','INSEE_COM','NOM_COMMUNE','SIREN_EPCI',
       'NOM_EPCI','NUM_PERMIS','DATE','ANNEE_MOIS_AUTORISATION','PERMIS_ANNULE','SUPERF_TERRAIN','DESTINATION',
       'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW','NB_LOGEMENTS_NEW','TYPE_HABIT_PRINCIP_SECOND','RESID_SECOND',
       'CODE_ACTIV_DEMANDEUR','CAT_JURID_DEMANDEUR','NOM_DEMANDEUR',
       'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR','DEP_DEMANDEUR','DEM_INSEE_COM','NOM_COMMUNE_DEMANDEUR','EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE','DEM_MEME_EPCI','DEM_MEME_DEP', 'DEM_BRETAGNE', 'DEM_Ouest', 'DEM_LOINTAINS',
       'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN', 'NOM_VOIE_TERRAIN',
       'CP_TERRAIN', 'SEC_CADASTRE1', 'NUM_CADASTRE1', 'SEC_CADASTRE2',
       'NUM_CADASTRE2', 'SEC_CADASTRE3', 'NUM_CADASTRE3','latitude','longitude','parcel_confidence','parcelle']]

Koumoul_loge.info()


/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/1413074225.py:1: DtypeWarning: Columns (22,30) have mixed types. Specify dtype option on import or set low_memory=False.
  Koumoul_loge=pd.read_csv("SITADEL/SITADEL_Koumoul_2014_2024/Koumoul_sitadel-logements.csv")
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/1413074225.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/1413074225.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Detail_Commune

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183206 entries, 0 to 183205
Data columns (total 46 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Type_AU                    183206 non-null  object        
 1   DEP                        183206 non-null  object        
 2   INSEE_COM                  183206 non-null  object        
 3   NOM_COMMUNE                183203 non-null  object        
 4   SIREN_EPCI                 183206 non-null  object        
 5   NOM_EPCI                   183170 non-null  object        
 6   NUM_PERMIS                 183206 non-null  object        
 7   DATE                       183206 non-null  object        
 8   ANNEE_MOIS_AUTORISATION    183206 non-null  datetime64[ns]
 9   PERMIS_ANNULE              183206 non-null  object        
 10  SUPERF_TERRAIN             183206 non-null  int64         
 11  DESTINATION                183206 non-null  int64   

Locaux Sitadel Koumoul :

In [4]:
# Chargement de la liste des permis de construire de locaux :
Koumoul_Loc=pd.read_csv("SITADEL/SITADEL_Koumoul_2014_2024/Koumoul_sitadel-locaux.csv", low_memory=False)
Koumoul_Loc["DATE_REELLE_AUTORISATION"]=Koumoul_Loc["DATE_REELLE_AUTORISATION"].astype("str")

Koumoul_Loc=Koumoul_Loc.rename(columns={"COMM":"INSEE_COM","DEP_CODE":"DEP","DATE_REELLE_AUTORISATION":"DATE","NUM_DAU":"NUM_PERMIS","ETAT_DAU":"ETAT_AVANCEMENT",
    "CODPOST_DEM":"CP_DEMANDEUR","LOCALITE_DEM":"COMMUNE_DEMANDEUR","ADR_NUM_TER":"NUM_VOIE_TERRAIN",
    "ADR_TYPEVOIE_TER":"TYPE_VOIE-TERRAIN","ADR_LIBVOIE_TER":"NOM_VOIE_TERRAIN","ADR_CODPOST_TER":"CP_TERRAIN",
    "ADR_LOCALITE_TER":"COMMUNE_TERRAIN","SUPERFICIE_TERRAIN":"SUPERF_TERRAIN","NATURE_PROJET_DECLAREE":"NATURE_PROJET",
    "NATURE_PROJET_COMPLETEE":"NATURE_PROJET_DETAIL","DESTINATION_PRINCIPALE":"DESTINATION","SURF_HAB_CREEE":"SURF_HABIT_NEW",
    "SURF_LOC_CREEE":"SURF_LOCAUX_NEW", "SURF_HEB_CREEE":"SURF_HOTEL_NEW", "SURF_BUR_CREEE":"SURF_BUREAU_NEW",
    "SURF_COM_CREEE":"SURF_COMMERC_NEW","SURF_ART_CREEE":"SURF_ARTISAN_NEW","SURF_IND_CREEE":"SURF_INDUS_NEW",
    "SURF_AGR_CREEE":"SURF_AGRIC_NEW","SURF_ENT_CREEE":"SURF_ENTREPOT_NEW","SURF_PUB_CREEE":"SURF_SERV_PUB_NEW",
    "TYPE_SERVICE_PUBLIC":"TYPE_SERV_PUB","APE_DEM":"CODE_ACTIV_DEMANDEUR","CAT_DEM":"CAT_JURID_DEMANDEUR",
    "DENOM_DEM":"NOM_DEMANDEUR","SIREN_DEM":"SIREN_DEMANDEUR","SIRET_DEM":"SIRET_DEMANDEUR"})

Koumoul_Loc["NATURE_PROJET"]=Koumoul_Loc["NATURE_PROJET"].astype("str")
Koumoul_Loc=Koumoul_Loc[Koumoul_Loc["NATURE_PROJET"]=="1"]

#### Création d'une nouvelle variable permettant de savoir si le permis a été annulé, 
# à partir du code fournir par la variable "ETAT_AVANCEMENT", qu'on supprime ensuite :
Koumoul_Loc.loc[Koumoul_Loc["ETAT_AVANCEMENT"].astype("str")=="4", "PERMIS_ANNULE"]="oui"
Koumoul_Loc.loc[Koumoul_Loc["ETAT_AVANCEMENT"].astype("str")!="4", "PERMIS_ANNULE"]="non"
Koumoul_Loc.drop(columns="ETAT_AVANCEMENT", inplace=True)

# Création d'une nouvelle variable "ANNEE_MOIS_AUTORISATION" pour préparer une synthèse mensuelle des permis :
Koumoul_Loc["DATE"]=Koumoul_Loc["DATE"].astype("str")
Koumoul_Loc["ANNEE_MOIS_AUTORISATION"]=(Koumoul_Loc["DATE"].str[:7])+"-01 01:00:00"
Koumoul_Loc["ANNEE_MOIS_AUTORISATION"]= pd.to_datetime(Koumoul_Loc["ANNEE_MOIS_AUTORISATION"])

# Modifier les codes insee des communes (qui ont pu changer depuis 2013 suite aux fusions de communes), 
# en utilisant la table de passage (https://www.insee.fr/fr/information/7671867):
Koumoul_Loc["INSEE_COM"]=Koumoul_Loc["INSEE_COM"].astype("str")
Koumoul_Loc.replace({"INSEE_COM": {"22154":"22093", "22173":"22093", "22038":"22206", "22100":"22219", "22078":"22264", "22247":"22264", "22253":"22264", "22123":"22050", "22007":"22055", "22167":"22107", "22290":"22107", "22298":"22158", "22192":"22209", "22357":"22209", "22066":"22046", "22102":"22046", "22191":"22046", "22292":"22046", "22297":"22046", "22303":"22046", "22051":"22084", "22151":"22093", "22058":"22183", "22080":"22203", "22367":"22251", "29219":"29199", "29129":"29227", "29203":"29021", "29149":"29076", "29052":"29003", "29127":"29266", "35112":"35062", "35113":"35004", "35303":"35004", "35341":"35004", "35100":"35163", "35301":"35184", "35053":"35220", "35269":"35282", "35293":"35282", "35348":"35282", "35011":"35292", "35147":"35308", "35344":"35308", "35209":"35069", "35254":"35069", "35048":"35168", "35083":"35191", "35323":"35191", "35267":"35257", "35158":"35060", "35129":"35176", "44160":"44003", "44017":"44180", "44093":"44180", "44191":"44180", "44219":"44180", "44040":"44005", "44059":"44021", "44008":"44029", "44181":"44087", "44004":"44163", "44011":"44213", "44034":"44213", "44147":"44213", "44060":"49160", "56049":"56213", "56059":"56102", "56138":"56165", "56016":"56173", "56183":"56033", "56038":"56061", "56064":"56061", "56142":"56144", "56192":"56144", "56037":"56197", "56187":"56197", "56150":"56251"}}, inplace=True)

# Ajout du statut et de l'Epci de rattachement des communes :
Koumoul_Loc["INSEE_COM"]=Koumoul_Loc["INSEE_COM"].astype("str")
Com_BZH=Com_BZH[["INSEE_COM","NOM_COMMUNE","SIREN_EPCI","NOM_EPCI"]]
Com_BZH["INSEE_COM"]=Com_BZH["INSEE_COM"].astype("str")
Koumoul_Loc=Koumoul_Loc.merge(Com_BZH, how="left", on="INSEE_COM")
Koumoul_Loc.rename(columns={"DEP_x":"DEP"}, inplace=True)


#  Faute d'un code insee pour la commune du demandeur, 
# on doit le recupérer en croisant le nom de commune avec la liste des propre des communes bretonnes, 
# et en profiter pour ajouter le code de l'EPCI et la statut (littoral...)

# On extrait une liste propre des communes :
Detail_Communes=Koumoul_Loc[["COMMUNE_TERRAIN", "NOM_COMMUNE", "SIREN_EPCI","INSEE_COM"]]
Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
Detail_Communes.sort_values(by="COMMUNE_TERRAIN", inplace=True)
Detail_Communes.rename(columns={"COMMUNE_TERRAIN":"COMMUNE_DEMANDEUR", "NOM_COMMUNE":"NOM_COMMUNE_ok", "SIREN_EPCI":"EPCI_DEMANDEUR", "INSEE_COM":"DEM_INSEE_COM"}, inplace=True)

# Puis on fusionne avec la base Sitadel :
Koumoul_Loc=Koumoul_Loc.merge(Detail_Communes, how="left", on="COMMUNE_DEMANDEUR")
Koumoul_Loc.replace({"EPCI_DEMANDEUR": {"NR": "0"}}, inplace=True)
Koumoul_Loc["EPCI_DEMANDEUR"]=Koumoul_Loc["EPCI_DEMANDEUR"].fillna("0").astype("int64")

# Le demandeur du permis est-il domicilié dans la même commune ?
Koumoul_Loc.loc[Koumoul_Loc["COMMUNE_TERRAIN"]==Koumoul_Loc["COMMUNE_DEMANDEUR"],"DEM_MEME_COMMUNE"]="1"
Koumoul_Loc["DEM_MEME_COMMUNE"]=Koumoul_Loc["DEM_MEME_COMMUNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le même département ?
Koumoul_Loc["DEP_DEMANDEUR"]=Koumoul_Loc["CP_DEMANDEUR"].str[:2]
Koumoul_Loc["DEP_DEMANDEUR"]=Koumoul_Loc["DEP_DEMANDEUR"].astype("str")
Koumoul_Loc["DEP"]=Koumoul_Loc["DEP"].astype("str")
Koumoul_Loc["DEP"]=Koumoul_Loc["DEP"].str[:2]
Koumoul_Loc.loc[Koumoul_Loc["DEP"]==Koumoul_Loc["DEP_DEMANDEUR"] ,"DEM_MEME_DEP"]="1"
Koumoul_Loc["DEM_MEME_DEP"]=Koumoul_Loc["DEM_MEME_DEP"].fillna("0")

# Le demandeur est-il du permis est domicilié en Bretagne ? 
dep_Bzh=("29","22","56","35")
Koumoul_Loc.loc[Koumoul_Loc["DEP_DEMANDEUR"].isin(dep_Bzh),"DEM_BRETAGNE"]="1"
Koumoul_Loc["DEM_BRETAGNE"]=Koumoul_Loc["DEM_BRETAGNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le quart nord-ouest de la France (Bretagne, Pays de la Loire ou ex-basse-Normandie) ?
Ouest=("44","85","53","72","49","14","50","61","29","22","56","35")
Koumoul_Loc.loc[Koumoul_Loc["DEP_DEMANDEUR"].isin(Ouest),"DEM_Ouest"]="1"
Koumoul_Loc["DEM_Ouest"]=Koumoul_Loc["DEM_Ouest"].fillna("0")

# Le demandeur du permis est-il domicilié dans une autre région (autre que Bretagne, Pays de la Loire ou Normandie) ?
Koumoul_Loc.loc[Koumoul_Loc["DEM_Ouest"]=="0","DEM_LOINTAINS"]="1"
Koumoul_Loc["DEM_LOINTAINS"]=Koumoul_Loc["DEM_LOINTAINS"].fillna("0")

# Le demandeur est-il du permis est domicilié dans le même EPCI ?
Koumoul_Loc["SIREN_EPCI"]=Koumoul_Loc["SIREN_EPCI"].astype("str")
Koumoul_Loc["EPCI_DEMANDEUR"]=Koumoul_Loc["EPCI_DEMANDEUR"].astype("str")
Koumoul_Loc.loc[Koumoul_Loc["SIREN_EPCI"]==Koumoul_Loc["EPCI_DEMANDEUR"],"DEM_MEME_EPCI"]="1"
Koumoul_Loc["DEM_MEME_EPCI"]=Koumoul_Loc["DEM_MEME_EPCI"].fillna("0")

Koumoul_Loc.rename(columns={"NOM_COMMUNE_ok":"NOM_COMMUNE_DEMANDEUR"}, inplace=True)
Koumoul_Loc.drop(columns={"COMMUNE_DEMANDEUR","COMMUNE_TERRAIN"}, inplace=True)

Koumoul_Loc["Type_AU"]="Locaux"

Koumoul_Loc=Koumoul_Loc[['Type_AU','DEP','INSEE_COM','NOM_COMMUNE','SIREN_EPCI',
       'NOM_EPCI','NUM_PERMIS','DATE','ANNEE_MOIS_AUTORISATION','PERMIS_ANNULE','SUPERF_TERRAIN',
       'NATURE_PROJET','NATURE_PROJET_DETAIL', 'DESTINATION',
       'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW', 'SURF_HOTEL_NEW','SURF_BUREAU_NEW', 'SURF_COMMERC_NEW', 'SURF_ARTISAN_NEW',
       'SURF_INDUS_NEW', 'SURF_AGRIC_NEW', 'SURF_ENTREPOT_NEW','SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB', 
       'CODE_ACTIV_DEMANDEUR','NOM_DEMANDEUR',
       'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR','DEP_DEMANDEUR','DEM_INSEE_COM','NOM_COMMUNE_DEMANDEUR','EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE','DEM_MEME_EPCI','DEM_MEME_DEP', 'DEM_BRETAGNE', 'DEM_Ouest', 'DEM_LOINTAINS',
       'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN', 'NOM_VOIE_TERRAIN',
       'CP_TERRAIN', 'SEC_CADASTRE1', 'NUM_CADASTRE1', 'SEC_CADASTRE2',
       'NUM_CADASTRE2', 'SEC_CADASTRE3', 'NUM_CADASTRE3','latitude','longitude','parcel_confidence','parcelle']]

Koumoul_Loc.info()


/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/362191271.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/362191271.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Detail_Communes.sort_values(by="COMMUNE_TERRAIN", inplace=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/362191271.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35310 entries, 0 to 35309
Data columns (total 53 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Type_AU                  35310 non-null  object        
 1   DEP                      35310 non-null  object        
 2   INSEE_COM                35310 non-null  object        
 3   NOM_COMMUNE              35309 non-null  object        
 4   SIREN_EPCI               35310 non-null  object        
 5   NOM_EPCI                 35284 non-null  object        
 6   NUM_PERMIS               35310 non-null  object        
 7   DATE                     35310 non-null  object        
 8   ANNEE_MOIS_AUTORISATION  35310 non-null  datetime64[ns]
 9   PERMIS_ANNULE            35310 non-null  object        
 10  SUPERF_TERRAIN           35310 non-null  int64         
 11  NATURE_PROJET            35310 non-null  object        
 12  NATURE_PROJET_DETAIL     35310 n

In [5]:
# Concat des deux fichiers Koumoul
Koumoul_Bzh=pd.concat([Koumoul_loge,Koumoul_Loc])
Koumoul_Bzh=Koumoul_Bzh.reset_index()

Koumoul_Bzh.replace({"SIREN_EPCI": {"nan": "0", "NR":"0"}}, inplace=True)
Koumoul_Bzh["SIREN_EPCI"]=Koumoul_Bzh["SIREN_EPCI"].fillna("0").astype("int64")
Koumoul_Bzh["SURF_HABIT_NEW"]=Koumoul_Bzh["SURF_HABIT_NEW"].fillna("0").astype("int64")
Koumoul_Bzh["SURF_HOTEL_NEW"]=Koumoul_Bzh["SURF_HOTEL_NEW"].fillna("0").astype("int64")
Koumoul_Bzh["SURF_BUREAU_NEW"]=Koumoul_Bzh["SURF_BUREAU_NEW"].fillna("0").astype("int64")
Koumoul_Bzh["SURF_COMMERC_NEW"]=Koumoul_Bzh["SURF_COMMERC_NEW"].fillna("0").astype("int64")
Koumoul_Bzh["SURF_ARTISAN_NEW"]=Koumoul_Bzh["SURF_ARTISAN_NEW"].fillna("0").astype("int64")
Koumoul_Bzh["SURF_INDUS_NEW"]=Koumoul_Bzh["SURF_INDUS_NEW"].fillna("0").astype("int64")
Koumoul_Bzh["SURF_ENTREPOT_NEW"]=Koumoul_Bzh["SURF_ENTREPOT_NEW"].fillna("0").astype("int64")
Koumoul_Bzh["SURF_SERV_PUB_NEW"]=Koumoul_Bzh["SURF_SERV_PUB_NEW"].fillna("0").astype("int64")
Koumoul_Bzh["RESID_SECOND"]=Koumoul_Bzh["RESID_SECOND"].fillna("0").astype("int64")
Koumoul_Bzh["SIREN_DEMANDEUR"]=Koumoul_Bzh["SIREN_DEMANDEUR"].fillna("0").astype("int64")
Koumoul_Bzh["SIRET_DEMANDEUR"]=Koumoul_Bzh["SIRET_DEMANDEUR"].fillna("0").astype("int64")
Koumoul_Bzh["TYPE_SERV_PUB"]=Koumoul_Bzh["TYPE_SERV_PUB"].fillna("0").astype("int64")
Koumoul_Bzh["CAT_JURID_DEMANDEUR"]=Koumoul_Bzh["CAT_JURID_DEMANDEUR"].fillna("0").astype("int64")
Koumoul_Bzh["NB_LOGEMENTS_NEW"]=Koumoul_Bzh["NB_LOGEMENTS_NEW"].fillna("0").astype("int64")
Koumoul_Bzh["TYPE_HABIT_PRINCIP_SECOND"]=Koumoul_Bzh["TYPE_HABIT_PRINCIP_SECOND"].fillna("0").astype("int64")
Koumoul_Bzh["SURF_AGRIC_NEW"]=Koumoul_Bzh["SURF_AGRIC_NEW"].fillna("0").astype("int64")
Koumoul_Bzh["NATURE_PROJET_DETAIL"]=Koumoul_Bzh["NATURE_PROJET_DETAIL"].fillna("0").astype("int64")

Koumoul_Bzh.to_csv("SITADEL/Koumoul_BZH_2014_2024.csv")
Koumoul_Bzh_geo=gpd.GeoDataFrame(Koumoul_Bzh, geometry=gpd.points_from_xy(Koumoul_Bzh.longitude, Koumoul_Bzh.latitude), crs="EPSG:4326")
Koumoul_Bzh_geo.to_file("SITADEL/Koumoul_BZH_2014_2024.geojson")
Koumoul_Bzh_geo.info()



/Users/denisvannier/opt/anaconda3/envs/GEODATA2/lib/python3.9/site-packages/pyogrio/geopandas.py:523: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  has_z_arr = geometry[geometry.notna() & (~geometry.is_empty)].has_z


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 218516 entries, 0 to 218515
Data columns (total 59 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   index                      218516 non-null  int64         
 1   Type_AU                    218516 non-null  object        
 2   DEP                        218516 non-null  object        
 3   INSEE_COM                  218516 non-null  object        
 4   NOM_COMMUNE                218512 non-null  object        
 5   SIREN_EPCI                 218516 non-null  int64         
 6   NOM_EPCI                   218454 non-null  object        
 7   NUM_PERMIS                 218516 non-null  object        
 8   DATE                       218516 non-null  object        
 9   ANNEE_MOIS_AUTORISATION    218516 non-null  datetime64[ns]
 10  PERMIS_ANNULE              218516 non-null  object        
 11  SUPERF_TERRAIN             218516 non-null  

### Traitement de la base SITADEL brute

#### Logements

In [109]:
SITADEL_loge=pd.read_csv("SITADEL/SITADEL/Liste-des-autorisations-durbanisme-creant-des-logements.2024-12.csv", sep=";")
SITADEL_loge["Code du département du lieu des travaux - Code de la zone"]=SITADEL_loge["Code du département du lieu des travaux - Code de la zone"].astype("str")
SITADEL_loge=SITADEL_loge[
    (SITADEL_loge["Code du département du lieu des travaux - Code de la zone"]=="44") | 
    (SITADEL_loge["Code du département du lieu des travaux - Code de la zone"]=="22") | 
    (SITADEL_loge["Code du département du lieu des travaux - Code de la zone"]=="29") | 
    (SITADEL_loge["Code du département du lieu des travaux - Code de la zone"]=="35") | 
    (SITADEL_loge["Code du département du lieu des travaux - Code de la zone"]=="56")]

SITADEL_loge.rename(columns={"Code de la commune du lieu des travaux":"INSEE_COM",
                            "Date réelle d’autorisation (PC) ou de non-opposition (DP) initiale":"DATE",
                            "Numéro d’enregistrement de la DAU":"NUM_PERMIS",
                            "Etat d’avancement du projet":"ETAT_AVANCEMENT",
                            "Code postal du demandeur":"CP_DEMANDEUR",
                            "Localité du demandeur":"COMMUNE_DEMANDEUR",
                            "Numéro de voie du terrain":"NUM_VOIE_TERRAIN",
                            "Type de voie du terrain":"TYPE_VOIE-TERRAIN",
                            "Libellé de la voie du terrain":"NOM_VOIE_TERRAIN",
                            "Code postal du terrain":"CP_TERRAIN",
                            "Localité du terrain":"COMMUNE_TERRAIN",
                            "Superficie du terrain":"SUPERF_TERRAIN",
                            "Nature du projet déclarée par le demandeur":"NATURE_PROJET", 
                            "Nature de projet détaillée":"NATURE_PROJET_DETAIL", 
                            "Destination principale":"DESTINATION", 
                            "Surface de plancher de la destination'Habitation'nouvelle construite":"SURF_HABIT_NEW",
                            "Surface de plancher de locaux non résidentiels (i.e. autres que d’habitation) nouvelle construite":"SURF_LOCAUX_NEW",
                            "Nombre total de logements créés":"NB_LOGEMENTS_NEW",
                            "Type d’habitation principale ou secondaire":"TYPE_HABIT_PRINCIP_SECOND",
                            "Code d’activité principale de l’établissement d’un demandeur avéré en tant que personne morale":"CODE_ACTIV_DEMANDEUR", 
                            "Catégorie du demandeur (maître d’ouvrage) selon Sitadel":"CAT_JURID_DEMANDEUR", 
                            "Dénomination d’un demandeur avéré en tant que personne morale":"NOM_DEMANDEUR", 
                            "Numéro SIREN d’un demandeur avéré en tant que personne morale":"SIREN_DEMANDEUR", 
                            "Numéro SIRET d’un demandeur avéré en tant que personne morale":"SIRET_DEMANDEUR"
                            }, inplace=True)

# On ne retient que les nouvelles constructions : 
SITADEL_loge["NATURE_PROJET"]=SITADEL_loge["NATURE_PROJET"].astype("str")
SITADEL_loge=SITADEL_loge[SITADEL_loge["NATURE_PROJET"]=="1"]

#### Création d'une nouvelle variable permettant de savoir si le permis a été annulé, 
# à partir du code fournir par la variable "ETAT_AVANCEMENT", qu'on supprime ensuite :
SITADEL_loge.loc[SITADEL_loge["ETAT_AVANCEMENT"].astype("str")=="4", "PERMIS_ANNULE"]="oui"
SITADEL_loge.loc[SITADEL_loge["ETAT_AVANCEMENT"].astype("str")!="4", "PERMIS_ANNULE"]="non"
SITADEL_loge.drop(columns="ETAT_AVANCEMENT", inplace=True)


#### Création d'une nouvelle variable permettant de savoir 
# si le demandeur a déclaré qu'il ferait du logement construit sa résidence secondaire :
SITADEL_loge.loc[SITADEL_loge["TYPE_HABIT_PRINCIP_SECOND"].astype("str")=="2", "RESID_SECOND"]="1"
SITADEL_loge["TYPE_HABIT_PRINCIP_SECOND"]=SITADEL_loge["TYPE_HABIT_PRINCIP_SECOND"].fillna("0")

# Création d'une nouvelle variable "ANNEE_MOIS_AUTORISATION" pour préparer une synthèse mensuelle des permis :
SITADEL_loge["ANNEE_MOIS_AUTORISATION"]=(SITADEL_loge["DATE"].str[:7])+"-01 01:00:00"
SITADEL_loge["ANNEE_MOIS_AUTORISATION"]= pd.to_datetime(SITADEL_loge["ANNEE_MOIS_AUTORISATION"])

# Ajout du statut et de l'Epci de rattachement des communes :
SITADEL_loge["INSEE_COM"]=SITADEL_loge["INSEE_COM"].astype("str")
Com_BZH["INSEE_COM"]=Com_BZH["INSEE_COM"].astype("str")
SITADEL_loge=SITADEL_loge.merge(Com_BZH, how="left", on="INSEE_COM")

#  Faute d'un code insee pour la commune du demandeur, 
# on doit le recupérer en croisant le nom de commune avec la liste des propre des communes bretonnes, 
# et en profiter pour ajouter le code de l'EPCI et la statut (littoral...)

# On extrait une liste propre des communes :
Detail_Communes=SITADEL_loge[["COMMUNE_TERRAIN", "NOM_COMMUNE", "SIREN_EPCI","INSEE_COM"]]
Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
Detail_Communes.sort_values(by="COMMUNE_TERRAIN", inplace=True)
Detail_Communes.rename(columns={"COMMUNE_TERRAIN":"COMMUNE_DEMANDEUR", "NOM_COMMUNE":"NOM_COMMUNE_ok", "SIREN_EPCI":"EPCI_DEMANDEUR", "INSEE_COM":"DEM_INSEE_COM"}, inplace=True)

# Puis on fusionne avec la base Sitadel :
SITADEL_loge=SITADEL_loge.merge(Detail_Communes, how="left", on="COMMUNE_DEMANDEUR")

# Le demandeur du permis est-il domicilié dans la même commune ?
SITADEL_loge.loc[SITADEL_loge["COMMUNE_TERRAIN"]==SITADEL_loge["COMMUNE_DEMANDEUR"],"DEM_MEME_COMMUNE"]="1"
SITADEL_loge["DEM_MEME_COMMUNE"]=SITADEL_loge["DEM_MEME_COMMUNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le même département ?
SITADEL_loge["DEP_DEMANDEUR"]=SITADEL_loge["CP_DEMANDEUR"].str[:2]
SITADEL_loge["DEP_DEMANDEUR"]=SITADEL_loge["DEP_DEMANDEUR"].astype("str")
SITADEL_loge["INSEE_COM"]=SITADEL_loge["INSEE_COM"].astype("str")
SITADEL_loge["DEP"]=SITADEL_loge["INSEE_COM"].str[:2]
SITADEL_loge.loc[SITADEL_loge["DEP"]==SITADEL_loge["DEP_DEMANDEUR"] ,"DEM_MEME_DEP"]="1"
SITADEL_loge["DEM_MEME_DEP"]=SITADEL_loge["DEM_MEME_DEP"].fillna("0")

# Le demandeur est-il du permis est domicilié en Bretagne ? 
dep_Bzh=("29","22","56","35","44")
SITADEL_loge.loc[SITADEL_loge["DEP_DEMANDEUR"].isin(dep_Bzh),"DEM_BRETAGNE"]="1"
SITADEL_loge["DEM_BRETAGNE"]=SITADEL_loge["DEM_BRETAGNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le quart nord-ouest de la France (Bretagne, Pays de la Loire ou ex-basse-Normandie) ?
Ouest=("44","85","53","72","49","14","50","61","29","22","56","35")
SITADEL_loge.loc[SITADEL_loge["DEP_DEMANDEUR"].isin(Ouest),"DEM_Ouest"]="1"
SITADEL_loge["DEM_Ouest"]=SITADEL_loge["DEM_Ouest"].fillna("0")

# Le demandeur du permis est-il domicilié dans une autre région (autre que Bretagne, Pays de la Loire ou Normandie) ?
SITADEL_loge.loc[SITADEL_loge["DEM_Ouest"]=="0","DEM_LOINTAINS"]="1"
SITADEL_loge["DEM_LOINTAINS"]=SITADEL_loge["DEM_LOINTAINS"].fillna("0")

# Le demandeur est-il du permis est domicilié dans le même EPCI ?
SITADEL_loge.loc[SITADEL_loge["SIREN_EPCI"]==SITADEL_loge["EPCI_DEMANDEUR"],"DEM_MEME_EPCI"]="1"
SITADEL_loge["DEM_MEME_EPCI"]=SITADEL_loge["DEM_MEME_EPCI"].fillna("0")

SITADEL_loge.rename(columns={"NOM_COMMUNE_ok":"NOM_COMMUNE_DEMANDEUR"}, inplace=True)
SITADEL_loge.drop(columns={"COMMUNE_DEMANDEUR","COMMUNE_TERRAIN"}, inplace=True)

SITADEL_loge["SIREN_EPCI"]=SITADEL_loge["SIREN_EPCI"].fillna("0")
SITADEL_loge.replace({"SIREN_EPCI": {"NR": "0"}}, inplace=True)
SITADEL_loge["SIREN_EPCI"]=SITADEL_loge["SIREN_EPCI"].astype("int64")

SITADEL_loge["Type_AU"]="Logements"

SITADEL_loge=SITADEL_loge[['Type_AU','DEP','INSEE_COM','NOM_COMMUNE','SIREN_EPCI',
       'NOM_EPCI','NUM_PERMIS','DATE','ANNEE_MOIS_AUTORISATION','PERMIS_ANNULE','SUPERF_TERRAIN','DESTINATION',
       'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW','NB_LOGEMENTS_NEW','TYPE_HABIT_PRINCIP_SECOND','RESID_SECOND',
       'CODE_ACTIV_DEMANDEUR','CAT_JURID_DEMANDEUR','NOM_DEMANDEUR',
       'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR','DEP_DEMANDEUR','DEM_INSEE_COM','NOM_COMMUNE_DEMANDEUR','EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE','DEM_MEME_EPCI','DEM_MEME_DEP', 'DEM_BRETAGNE', 'DEM_Ouest', 'DEM_LOINTAINS',
       'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN', 'NOM_VOIE_TERRAIN',
       'CP_TERRAIN', 'sec_cadastre1', 'num_cadastre1', 'sec_cadastre2',
       'num_cadastre2', 'sec_cadastre3', 'num_cadastre3']]

SITADEL_loge.info()

/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/201520532.py:1: DtypeWarning: Columns (0,2,4,7,11,16,18,20,21,22,24,30,34,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  SITADEL_loge=pd.read_csv("SITADEL/SITADEL/Liste-des-autorisations-durbanisme-creant-des-logements.2024-12.csv", sep=";")
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/201520532.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  SITADEL_loge["TYPE_HABIT_PRINCIP_SECOND"]=SITADEL_loge["TYPE_HABIT_PRINCIP_SECOND"].fillna("0")
/var/folders/zf/sc5qjn6x54b

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187317 entries, 0 to 187316
Data columns (total 42 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Type_AU                    187317 non-null  object        
 1   DEP                        187317 non-null  object        
 2   INSEE_COM                  187317 non-null  object        
 3   NOM_COMMUNE                186127 non-null  object        
 4   SIREN_EPCI                 187317 non-null  int64         
 5   NOM_EPCI                   186091 non-null  object        
 6   NUM_PERMIS                 187317 non-null  object        
 7   DATE                       187317 non-null  object        
 8   ANNEE_MOIS_AUTORISATION    187317 non-null  datetime64[ns]
 9   PERMIS_ANNULE              187317 non-null  object        
 10  SUPERF_TERRAIN             187317 non-null  object        
 11  DESTINATION                187317 non-null  object  

#### Locaux

In [113]:
# Chargement de la liste des permis de construire de locaux :
SITADEL_Loc=pd.read_csv("SITADEL/SITADEL/Liste-des-autorisations-durbanisme-creant-des-locaux-non-residentiels.2024-12.csv", sep=";")
SITADEL_Loc["Code du département du lieu des travaux - Code de la zone"]=SITADEL_Loc["Code du département du lieu des travaux - Code de la zone"].astype("str")
SITADEL_Loc=SITADEL_Loc[
    (SITADEL_Loc["Code du département du lieu des travaux - Code de la zone"]=="44") | 
    (SITADEL_Loc["Code du département du lieu des travaux - Code de la zone"]=="22") | 
    (SITADEL_Loc["Code du département du lieu des travaux - Code de la zone"]=="29") | 
    (SITADEL_Loc["Code du département du lieu des travaux - Code de la zone"]=="35") | 
    (SITADEL_Loc["Code du département du lieu des travaux - Code de la zone"]=="56")]

SITADEL_Loc.rename(columns={"Code de la commune du lieu des travaux":"INSEE_COM",
                            "Date réelle d'autorisation (PC) ou de non-opposition (DP) initiale":"DATE",
                            "Numéro d'enregistrement de la DAU":"NUM_PERMIS",
                            "Etat d'avancement du projet":"ETAT_AVANCEMENT",
                            "Code postal du demandeur":"CP_DEMANDEUR",
                            "Localité du demandeur":"COMMUNE_DEMANDEUR",
                            "Numéro de voie du terrain":"NUM_VOIE_TERRAIN",
                            "Type de voie du terrain":"TYPE_VOIE-TERRAIN",
                            "Libellé de la voie du terrain":"NOM_VOIE_TERRAIN",
                            "Code postal du terrain":"CP_TERRAIN",
                            "Localité du terrain":"COMMUNE_TERRAIN",
                            "Superficie du terrain":"SUPERF_TERRAIN",
                            "Nature du projet déclarée par le demandeur":"NATURE_PROJET", 
                            "Nature de projet détaillée":"NATURE_PROJET_DETAIL", 
                            "Destination principale":"DESTINATION", 
                            "Surface de plancher de la destination Habitation nouvelle construite":"SURF_HABIT_NEW", 
                            "Surface de plancher de locaux non résidentiels (i.e. autres que d'habitation) nouvelle construite":"SURF_LOCAUX_NEW", 
                            "Surface de plancher de la destination Hébergement hôtelier nouvelle construite":"SURF_HOTEL_NEW", 
                            "Surface de plancher de la destination Bureau nouvelle construite":"SURF_BUREAU_NEW", 
                            "Surface de plancher de la destination Commerce nouvelle construite":"SURF_COMMERC_NEW", 
                            "Surface de plancher de la destination Artisanat nouvelle construite":"SURF_ARTISAN_NEW", 
                            "Surface de plancher de la destination Industrie nouvelle construite":"SURF_INDUS_NEW", 
                            "Surface de plancher de la destination Exploitation agricole ou forestière nouvelle construite":"SURF_AGRIC_NEW", 
                            "Surface de plancher de la destination Entrepôt nouvelle construite":"SURF_ENTREPOT_NEW", 
                            "Surface de plancher de la destination Services publics ou d'intérêt collectif nouvelle construite":"SURF_SERV_PUB_NEW", 
                            "Type de service public ou d'intérêt collectif":"TYPE_SERV_PUB", 
                            "Code d'activité principale de l'établissement d'un demandeur avéré en tant que personne morale":"CODE_ACTIV_DEMANDEUR", 
                            "Catégorie juridique d'un demandeur avéré en tant que personne morale":"CAT_JURID_DEMANDEUR", 
                            "Dénomination d'un demandeur avéré en tant que personne morale":"NOM_DEMANDEUR", 
                            "Numéro SIREN d'un demandeur avéré en tant que personne morale":"SIREN_DEMANDEUR", 
                            "Numéro SIRET d'un demandeur avéré en tant que personne morale":"SIRET_DEMANDEUR"
                            }, inplace=True)

SITADEL_Loc["NATURE_PROJET"]=SITADEL_Loc["NATURE_PROJET"].astype("str")
SITADEL_Loc=SITADEL_Loc[SITADEL_Loc["NATURE_PROJET"]=="1"]

#### Création d'une nouvelle variable permettant de savoir si le permis a été annulé, 
# à partir du code fournir par la variable "ETAT_AVANCEMENT", qu'on supprime ensuite :
SITADEL_Loc.loc[SITADEL_Loc["ETAT_AVANCEMENT"].astype("str")=="4", "PERMIS_ANNULE"]="oui"
SITADEL_Loc.loc[SITADEL_Loc["ETAT_AVANCEMENT"].astype("str")!="4", "PERMIS_ANNULE"]="non"
SITADEL_Loc.drop(columns="ETAT_AVANCEMENT", inplace=True)

# Création d'une nouvelle variable "ANNEE_MOIS_AUTORISATION" pour préparer une synthèse mensuelle des permis :
SITADEL_Loc["ANNEE_MOIS_AUTORISATION"]=(SITADEL_Loc["DATE"].str[:7])+"-01 01:00:00"
SITADEL_Loc["ANNEE_MOIS_AUTORISATION"]= pd.to_datetime(SITADEL_Loc["ANNEE_MOIS_AUTORISATION"])

# Ajout du statut et de l'Epci de rattachement des communes :
SITADEL_Loc["INSEE_COM"]=SITADEL_Loc["INSEE_COM"].astype("str")
Com_BZH["INSEE_COM"]=Com_BZH["INSEE_COM"].astype("str")
SITADEL_Loc=SITADEL_Loc.merge(Com_BZH, how="left", on="INSEE_COM")

#  Faute d'un code insee pour la commune du demandeur, 
# on doit le recupérer en croisant le nom de commune avec la liste des propre des communes bretonnes, 
# et en profiter pour ajouter le code de l'EPCI et la statut (littoral...)

# On extrait une liste propre des communes :
Detail_Communes=SITADEL_Loc[["COMMUNE_TERRAIN", "NOM_COMMUNE", "SIREN_EPCI","INSEE_COM"]]
Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
Detail_Communes.sort_values(by="COMMUNE_TERRAIN", inplace=True)
Detail_Communes.rename(columns={"COMMUNE_TERRAIN":"COMMUNE_DEMANDEUR", "NOM_COMMUNE":"NOM_COMMUNE_ok","SIREN_EPCI":"EPCI_DEMANDEUR", "INSEE_COM":"DEM_INSEE_COM"}, inplace=True)

# Puis on fusionne avec la base Sitadel :
SITADEL_Loc=SITADEL_Loc.merge(Detail_Communes, how="left", on="COMMUNE_DEMANDEUR")

# Le demandeur du permis est-il domicilié dans la même commune ?
SITADEL_Loc.loc[SITADEL_Loc["COMMUNE_TERRAIN"]==SITADEL_Loc["COMMUNE_DEMANDEUR"],"DEM_MEME_COMMUNE"]="1"
SITADEL_Loc["DEM_MEME_COMMUNE"]=SITADEL_Loc["DEM_MEME_COMMUNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le même département ?
SITADEL_Loc["DEP_DEMANDEUR"]=SITADEL_Loc["CP_DEMANDEUR"].str[:2]
SITADEL_Loc["DEP_DEMANDEUR"]=SITADEL_Loc["DEP_DEMANDEUR"].astype("str")
SITADEL_Loc["DEP"]=SITADEL_Loc["INSEE_COM"].str[:2]
SITADEL_Loc.loc[SITADEL_Loc["DEP"]==SITADEL_Loc["DEP_DEMANDEUR"] ,"DEM_MEME_DEP"]="1"
SITADEL_Loc["DEM_MEME_DEP"]=SITADEL_Loc["DEM_MEME_DEP"].fillna("0")

# Le demandeur est-il du permis est domicilié en Bretagne ? 
dep_Bzh=("29","22","56","35")
SITADEL_Loc.loc[SITADEL_Loc["DEP_DEMANDEUR"].isin(dep_Bzh),"DEM_BRETAGNE"]="1"
SITADEL_Loc["DEM_BRETAGNE"]=SITADEL_Loc["DEM_BRETAGNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le quart nord-ouest de la France (Bretagne, Pays de la Loire ou ex-basse-Normandie) ?
Ouest=("44","85","53","72","49","14","50","61","29","22","56","35")
SITADEL_Loc.loc[SITADEL_Loc["DEP_DEMANDEUR"].isin(Ouest),"DEM_Ouest"]="1"
SITADEL_Loc["DEM_Ouest"]=SITADEL_Loc["DEM_Ouest"].fillna("0")

# Le demandeur du permis est-il domicilié dans une autre région (autre que Bretagne, Pays de la Loire ou Normandie) ?
SITADEL_Loc.loc[SITADEL_Loc["DEM_Ouest"]=="0","DEM_LOINTAINS"]="1"
SITADEL_Loc["DEM_LOINTAINS"]=SITADEL_Loc["DEM_LOINTAINS"].fillna("0")

# Le demandeur est-il du permis est domicilié dans le même EPCI ?
SITADEL_Loc.loc[SITADEL_Loc["SIREN_EPCI"]==SITADEL_Loc["EPCI_DEMANDEUR"],"DEM_MEME_EPCI"]="1"
SITADEL_Loc["DEM_MEME_EPCI"]=SITADEL_Loc["DEM_MEME_EPCI"].fillna("0")

SITADEL_Loc.rename(columns={"NOM_COMMUNE_ok":"NOM_COMMUNE_DEMANDEUR"}, inplace=True)
SITADEL_Loc.drop(columns={"COMMUNE_DEMANDEUR","COMMUNE_TERRAIN"}, inplace=True)

SITADEL_Loc["SIREN_EPCI"]=SITADEL_Loc["SIREN_EPCI"].fillna("0")
SITADEL_Loc.replace({"SIREN_EPCI": {"NR": "0"}}, inplace=True)
SITADEL_Loc["SIREN_EPCI"]=SITADEL_Loc["SIREN_EPCI"].astype("int64")

SITADEL_Loc["Type_AU"]="Locaux"

SITADEL_Loc=SITADEL_Loc[['Type_AU','DEP','INSEE_COM','NOM_COMMUNE','SIREN_EPCI',
       'NOM_EPCI','NUM_PERMIS','DATE','ANNEE_MOIS_AUTORISATION','PERMIS_ANNULE','SUPERF_TERRAIN',
       'NATURE_PROJET','NATURE_PROJET_DETAIL', 'DESTINATION',
       'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW', 'SURF_HOTEL_NEW','SURF_BUREAU_NEW', 'SURF_COMMERC_NEW', 'SURF_ARTISAN_NEW',
       'SURF_INDUS_NEW', 'SURF_AGRIC_NEW', 'SURF_ENTREPOT_NEW','SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB', 
       'CODE_ACTIV_DEMANDEUR','CAT_JURID_DEMANDEUR','NOM_DEMANDEUR',
       'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR','DEP_DEMANDEUR','DEM_INSEE_COM','NOM_COMMUNE_DEMANDEUR','EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE','DEM_MEME_EPCI','DEM_MEME_DEP', 'DEM_BRETAGNE', 'DEM_Ouest', 'DEM_LOINTAINS',
       'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN', 'NOM_VOIE_TERRAIN',
       'CP_TERRAIN', 'sec_cadastre1', 'num_cadastre1', 'sec_cadastre2',
       'num_cadastre2', 'sec_cadastre3', 'num_cadastre3']]

SITADEL_Loc.info()


/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/3624961906.py:2: DtypeWarning: Columns (0,2,4,7,11,17,19,20,21,23,29,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98) have mixed types. Specify dtype option on import or set low_memory=False.
  SITADEL_Loc=pd.read_csv("SITADEL/SITADEL/Liste-des-autorisations-durbanisme-creant-des-locaux-non-residentiels.2024-12.csv", sep=";")
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/3624961906.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_80326/36249619

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36316 entries, 0 to 36315
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Type_AU                  36316 non-null  object        
 1   DEP                      36316 non-null  object        
 2   INSEE_COM                36316 non-null  object        
 3   NOM_COMMUNE              35963 non-null  object        
 4   SIREN_EPCI               36316 non-null  int64         
 5   NOM_EPCI                 35937 non-null  object        
 6   NUM_PERMIS               36316 non-null  object        
 7   DATE                     36316 non-null  object        
 8   ANNEE_MOIS_AUTORISATION  36316 non-null  datetime64[ns]
 9   PERMIS_ANNULE            36316 non-null  object        
 10  SUPERF_TERRAIN           36316 non-null  object        
 11  NATURE_PROJET            36316 non-null  object        
 12  NATURE_PROJET_DETAIL     36316 n

#### Fusion de Logements + Locaux

In [8]:
SITADEL_Bzh=pd.concat([SITADEL_loge,SITADEL_Loc])
SITADEL_Bzh=SITADEL_Bzh.reset_index()

# Reconstituer l'identifiant de parcelle cadastrale :
SITADEL_Bzh["sec_cadastre1"]=SITADEL_Bzh["sec_cadastre1"].astype("str")
SITADEL_Bzh.loc[SITADEL_Bzh["sec_cadastre1"].str.len()==1 , "Section_CADASTRE"]="0"+SITADEL_Bzh["sec_cadastre1"]
SITADEL_Bzh.loc[SITADEL_Bzh["sec_cadastre1"].str.len()==2 , "Section_CADASTRE"]=SITADEL_Bzh["sec_cadastre1"]
SITADEL_Bzh["num_cadastre1"]=SITADEL_Bzh["num_cadastre1"].astype("str")
SITADEL_Bzh.loc[SITADEL_Bzh["num_cadastre1"].str.len()==1 , "Numero_CADASTRE"]="000"+SITADEL_Bzh["num_cadastre1"]
SITADEL_Bzh.loc[SITADEL_Bzh["num_cadastre1"].str.len()==2 , "Numero_CADASTRE"]="00"+SITADEL_Bzh["num_cadastre1"]
SITADEL_Bzh.loc[SITADEL_Bzh["num_cadastre1"].str.len()==3 , "Numero_CADASTRE"]="0"+SITADEL_Bzh["num_cadastre1"]
SITADEL_Bzh.loc[SITADEL_Bzh["num_cadastre1"].str.len()==4 , "Numero_CADASTRE"]=SITADEL_Bzh["num_cadastre1"]
SITADEL_Bzh["NUM_CADASTRE"]=SITADEL_Bzh["INSEE_COM"]+"000"+SITADEL_Bzh["Section_CADASTRE"]+SITADEL_Bzh["Numero_CADASTRE"]

SITADEL_Bzh=SITADEL_Bzh[["Type_AU", "DEP", "INSEE_COM", "NOM_COMMUNE",
       "SIREN_EPCI", "NOM_EPCI", "NUM_PERMIS", "NUM_CADASTRE", "DATE",
       "ANNEE_MOIS_AUTORISATION", "PERMIS_ANNULE", "SUPERF_TERRAIN",
       "DESTINATION", "SURF_HABIT_NEW", "SURF_LOCAUX_NEW", "NB_LOGEMENTS_NEW",
       "TYPE_HABIT_PRINCIP_SECOND", "RESID_SECOND", "CODE_ACTIV_DEMANDEUR",
       "CAT_JURID_DEMANDEUR", "NOM_DEMANDEUR", "SIREN_DEMANDEUR",
       "SIRET_DEMANDEUR", "DEP_DEMANDEUR", "DEM_INSEE_COM",
       "NOM_COMMUNE_DEMANDEUR", "EPCI_DEMANDEUR",
       "DEM_MEME_COMMUNE", "DEM_MEME_EPCI", "DEM_MEME_DEP", "DEM_BRETAGNE",
       "DEM_Ouest", "DEM_LOINTAINS", "NUM_VOIE_TERRAIN", "TYPE_VOIE-TERRAIN",
       "NOM_VOIE_TERRAIN", "CP_TERRAIN", "sec_cadastre1", "num_cadastre1",
       "sec_cadastre2", "num_cadastre2", "sec_cadastre3", "num_cadastre3",
       "NATURE_PROJET", "NATURE_PROJET_DETAIL", "SURF_HOTEL_NEW",
       "SURF_BUREAU_NEW", "SURF_COMMERC_NEW", "SURF_ARTISAN_NEW",
       "SURF_INDUS_NEW", "SURF_AGRIC_NEW", "SURF_ENTREPOT_NEW",
       "SURF_SERV_PUB_NEW", "TYPE_SERV_PUB"]]

#### FILTRAGE DES PERMIS DELIVRES DEPUIS LA DERNIERE MISE A JOUR DE LA BASE KOUMOUL :
SITADEL_Bzh=SITADEL_Bzh[SITADEL_Bzh["ANNEE_MOIS_AUTORISATION"]>="2024-07-01 01:00:00"]

SITADEL_Bzh.info()



<class 'pandas.core.frame.DataFrame'>
Index: 4671 entries, 29 to 223617
Data columns (total 54 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Type_AU                    4671 non-null   object        
 1   DEP                        4671 non-null   object        
 2   INSEE_COM                  4671 non-null   object        
 3   NOM_COMMUNE                4671 non-null   object        
 4   SIREN_EPCI                 4671 non-null   int64         
 5   NOM_EPCI                   4667 non-null   object        
 6   NUM_PERMIS                 4671 non-null   object        
 7   NUM_CADASTRE               4609 non-null   object        
 8   DATE                       4671 non-null   object        
 9   ANNEE_MOIS_AUTORISATION    4671 non-null   datetime64[ns]
 10  PERMIS_ANNULE              4671 non-null   object        
 11  SUPERF_TERRAIN             4671 non-null   object        
 12  DESTINAT

#### Permis d'aménager

In [44]:
# Chargement de la liste des permis d’aménager (PA) :
SITADEL_Amenag=pd.read_csv("SITADEL/SITADEL/Liste-des-permis-damenager.2024-12.csv", sep=";")
SITADEL_Amenag["Code du département du lieu des travaux - Code de la zone"]=SITADEL_Amenag["Code du département du lieu des travaux - Code de la zone"].astype("str")
SITADEL_Amenag=SITADEL_Amenag[
    (SITADEL_Amenag["Code du département du lieu des travaux - Code de la zone"]=="44") | 
    (SITADEL_Amenag["Code du département du lieu des travaux - Code de la zone"]=="22") | 
    (SITADEL_Amenag["Code du département du lieu des travaux - Code de la zone"]=="29") | 
    (SITADEL_Amenag["Code du département du lieu des travaux - Code de la zone"]=="35") | 
    (SITADEL_Amenag["Code du département du lieu des travaux - Code de la zone"]=="56")]

SITADEL_Amenag=SITADEL_Amenag[["Code de la commune du lieu des travaux","Date réelle d’autorisation initiale",
"Numéro d’enregistrement du permis d’aménager (PA)",
"Etat d’avancement du projet",
"Code d’activité principale de l’établissement d’un demandeur avéré en tant que personne morale",
"Catégorie juridique d’un demandeur avéré en tant que personne morale",
"Dénomination d’un demandeur avéré en tant que personne morale",
"Numéro SIREN d’un demandeur avéré en tant que personne morale",
"Numéro SIRET d’un demandeur avéré en tant que personne morale",
"Code postal du demandeur",
"Localité du demandeur",
"Numéro de voie du terrain",
"Type de voie du terrain",
"Libellé de la voie du terrain",
"Code postal du terrain",
"Localité du terrain",
"sec_cadastre1",
"num_cadastre1",
"sec_cadastre2",
"num_cadastre2",
"sec_cadastre3",
"num_cadastre3",
"Superficie du terrain",
"Code zone opératoire"]]

SITADEL_Amenag=SITADEL_Amenag.rename(columns={"Code de la commune du lieu des travaux":"INSEE_COM","Date réelle d’autorisation initiale":"DATE","Numéro d’enregistrement du permis d’aménager (PA)":"NUM_PA","Etat d’avancement du projet":"ETAT_AVANCEMENT","Code d’activité principale de l’établissement d’un demandeur avéré en tant que personne morale":"CODE_ACTIV_DEMANDEUR","Catégorie juridique d’un demandeur avéré en tant que personne morale":"CAT_JURID_DEMANDEUR","Dénomination d’un demandeur avéré en tant que personne morale":"NOM_DEMANDEUR","Numéro SIREN d’un demandeur avéré en tant que personne morale":"SIREN_DEMANDEUR","Numéro SIRET d’un demandeur avéré en tant que personne morale":"SIRET_DEMANDEUR","Code postal du demandeur":"CP_DEMANDEUR","Localité du demandeur":"COMMUNE_DEMANDEUR","Numéro de voie du terrain":"NUM_VOIE_TERRAIN","Type de voie du terrain":"TYPE_VOIE-TERRAIN","Libellé de la voie du terrain":"NOM_VOIE_TERRAIN","Code postal du terrain":"CP_TERRAIN","Localité du terrain":"COMMUNE_TERRAIN","Superficie du terrain":"SUPERF_TERRAIN",})

#### Création d'une nouvelle variable permetant de savoir si le permis a été annulé, 
# à partir du code fournir par la variable "ETAT_AVANCEMENT", qu'on supprime ensuite :
SITADEL_Amenag.loc[SITADEL_Amenag["ETAT_AVANCEMENT"].astype("str")=="4", "PERMIS_ANNULE"]="oui"
SITADEL_Amenag.loc[SITADEL_Amenag["ETAT_AVANCEMENT"].astype("str")!="4", "PERMIS_ANNULE"]="non"
SITADEL_Amenag.drop(columns="ETAT_AVANCEMENT", inplace=True)

# Création d'une nouvelle variable "ANNEE_MOIS_AUTORISATION" pour préparer une synthèse mensuelle des permis :
SITADEL_Amenag["ANNEE_MOIS_AUTORISATION"]=(SITADEL_Amenag["DATE"].str[:7])+"-01 01:00:00"
SITADEL_Amenag["ANNEE_MOIS_AUTORISATION"]= pd.to_datetime(SITADEL_Amenag["ANNEE_MOIS_AUTORISATION"])

# Ajout du statut et de l'Epci de rattachement des communes :
SITADEL_Amenag=SITADEL_Amenag.merge(Com_BZH, on="INSEE_COM")

#  Faute d'un code insee pour la commune du demandeur, 
# on doit le recupérer en croisant le nom de commune avec la liste des propre des communes bretonnes, 
# et en profiter pour ajouter le code de l'EPCI et la statut (littoral...)

# On extrait une liste propre des communes :
Detail_Communes=SITADEL_Amenag[["COMMUNE_TERRAIN", "NOM_COMMUNE", "SIREN_EPCI","INSEE_COM"]]
Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
Detail_Communes.sort_values(by="COMMUNE_TERRAIN", inplace=True)
Detail_Communes.rename(columns={"COMMUNE_TERRAIN":"COMMUNE_DEMANDEUR", "NOM_COMMUNE":"NOM_COMMUNE_ok", "SIREN_EPCI":"EPCI_DEMANDEUR", "INSEE_COM":"DEM_INSEE_COM"}, inplace=True)

# Puis on merge avec la base Sitadel :
SITADEL_Amenag=SITADEL_Amenag.merge(Detail_Communes, on="COMMUNE_DEMANDEUR")

# !!!!!!!!!!!!!!!   FILTRAGE DES PERMIS DELIVRES DEPUIS LE 1er JUILLET 2024 (exclus de la version Koumoul) !!!!!!!!!! :
SITADEL_Amenag=SITADEL_Amenag[SITADEL_Amenag["ANNEE_MOIS_AUTORISATION"]>="2024-07-01 01:00:00"]

# Le demandeur du permis est-il domicilié dans la même commune ?
SITADEL_Amenag.loc[SITADEL_Amenag["COMMUNE_TERRAIN"]==SITADEL_Amenag["COMMUNE_DEMANDEUR"],"DEM_MEME_COMMUNE"]="1"
SITADEL_Amenag["DEM_MEME_COMMUNE"]=SITADEL_Amenag["DEM_MEME_COMMUNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le même département ?
SITADEL_Amenag["DEP_DEMANDEUR"]=SITADEL_Amenag["CP_DEMANDEUR"].str[:2]
SITADEL_Amenag["DEP_DEMANDEUR"]=SITADEL_Amenag["DEP_DEMANDEUR"].astype("str")
SITADEL_Amenag["DEP"]=SITADEL_Amenag["DEP"].astype("str")
SITADEL_Amenag["DEP"]=SITADEL_Amenag["DEP"].str[:2]
SITADEL_Amenag.loc[SITADEL_Amenag["DEP"]==SITADEL_Amenag["DEP_DEMANDEUR"] ,"DEM_MEME_DEP"]="1"
SITADEL_Amenag["DEM_MEME_DEP"]=SITADEL_Amenag["DEM_MEME_DEP"].fillna("0")

# Le demandeur est-il du permis est domicilié en Bretagne ? 
dep_Bzh=("29","22","56","35")
SITADEL_Amenag.loc[SITADEL_Amenag["DEP_DEMANDEUR"].isin(dep_Bzh),"DEM_BRETAGNE"]="1"
SITADEL_Amenag["DEM_BRETAGNE"]=SITADEL_Amenag["DEM_BRETAGNE"].fillna("0")

# Le demandeur du permis est-il domicilié dans le quart nord-ouest de la France (Bretagne, Pays de la Loire ou ex-basse-Normandie) ?
Ouest=("44","85","53","72","49","14","50","61","29","22","56","35")
SITADEL_Amenag.loc[SITADEL_Amenag["DEP_DEMANDEUR"].isin(Ouest),"DEM_Ouest"]="1"
SITADEL_Amenag["DEM_Ouest"]=SITADEL_Amenag["DEM_Ouest"].fillna("0")

# Le demandeur du permis est-il domicilié dans une autre région (autre que Bretagne, Pays de la Loire ou Normandie) ?
SITADEL_Amenag.loc[SITADEL_Amenag["DEM_Ouest"]=="0","DEM_LOINTAINS"]="1"
SITADEL_Amenag["DEM_LOINTAINS"]=SITADEL_Amenag["DEM_LOINTAINS"].fillna("0")

# Le demandeur est-il du permis est domicilié dans le même EPCI ?
SITADEL_Amenag.loc[SITADEL_Amenag["SIREN_EPCI"]==SITADEL_Amenag["EPCI_DEMANDEUR"],"DEM_MEME_EPCI"]="1"
SITADEL_Amenag["DEM_MEME_EPCI"]=SITADEL_Amenag["DEM_MEME_EPCI"].fillna("0")

SITADEL_Amenag.rename(columns={"NOM_COMMUNE_ok":"NOM_COMMUNE_DEMANDEUR"}, inplace=True)
SITADEL_Amenag.drop(columns={"COMMUNE_DEMANDEUR","COMMUNE_TERRAIN"}, inplace=True)

SITADEL_Amenag["SIREN_EPCI"]=SITADEL_Amenag["SIREN_EPCI"].fillna("0")
SITADEL_Amenag.replace({"SIREN_EPCI": {"NR": "0"}}, inplace=True)
SITADEL_Amenag["SIREN_EPCI"]=SITADEL_Amenag["SIREN_EPCI"].astype("int64")

SITADEL_Amenag["Type_AU"]="Amenag"

SITADEL_Amenag=SITADEL_Amenag[['Type_AU','DEP','INSEE_COM','NOM_COMMUNE','SIREN_EPCI',
       'NOM_EPCI','DATE','ANNEE_MOIS_AUTORISATION','PERMIS_ANNULE','SUPERF_TERRAIN','Code zone opératoire',
       'CODE_ACTIV_DEMANDEUR','CAT_JURID_DEMANDEUR','NOM_DEMANDEUR',
       'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR','DEP_DEMANDEUR','DEM_INSEE_COM','NOM_COMMUNE_DEMANDEUR','EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE','DEM_MEME_EPCI','DEM_MEME_DEP', 'DEM_BRETAGNE', 'DEM_Ouest', 'DEM_LOINTAINS',
       'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN', 'NOM_VOIE_TERRAIN',
       'CP_TERRAIN', 'sec_cadastre1', 'num_cadastre1', 'sec_cadastre2',
       'num_cadastre2', 'sec_cadastre3', 'num_cadastre3', ]]

SITADEL_Amenag.sample(5)
SITADEL_Amenag.info()

/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_37192/259502933.py:2: DtypeWarning: Columns (0,2,4,6,10,16,18,19,22,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  SITADEL_Amenag=pd.read_csv("SITADEL/SITADEL/Liste-des-permis-damenager.2024-12.csv", sep=";")
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_37192/259502933.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Detail_Communes.drop_duplicates(subset=["COMMUNE_TERRAIN"], inplace=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_37192/259502933.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

KeyError: 'DEP'

#### Géocodage

In [9]:
#### CHARGEMENT DU CADASTRE géolocalisé par Etalab (millésime 1er janvier 2024) : https://cadastre.data.gouv.fr/datasets/cadastre-etalab
# Compilation de l'ensemble des millésimes disponibles (2017-2024) et filtré sur la Bretagne.
Cadastre_Geoloc_Bzh=gpd.read_file("Cadastre_Geoloc_2017-2024_BZH.geojson")
Cadastre_Geoloc_Bzh.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7409343 entries, 0 to 7409342
Data columns (total 4 columns):
 #   Column              Dtype   
---  ------              -----   
 0   NUM_CADASTRE        object  
 1   millesime_Cadastre  object  
 2   contenance          float64 
 3   geometry            geometry
dtypes: float64(1), geometry(1), object(2)
memory usage: 226.1+ MB


In [10]:
####### GEOCODAGE (Cadastre + BAN) #####
# Dans plusieurs communes, des fusions ont conduit à modifier les préfixes des parcelles, 
# qui ne sont plus "000" mais correspondent aux codes des communes absorbées 
# (ex : 123 pour une commune dont le code Insee était 22123). 
# Il n'est donc pas possible de faire correspondre les numéros de parcelles ,
# reconstitués de Sitadel avec ceux du Cadastre Etalab. 
# On liste donc ces communes pour procéder à une géolocalisation des adresses 
# en les soumettant à l'API de la Base Adresses Nationale (BAN). 
# En combinant un géocodage via le Cadastre Etalab puis la Base Adresse, on parvient à géolocaliser 91,5% des permis bretons. 


# Fusion de la base Sitadel avec le cadastre en utilisant le numéro de parcelle complet (en supposant un préfixe "000")
SITADEL_Bzh_geoloc_A=SITADEL_Bzh.merge(Cadastre_Geoloc_Bzh, how="left", on="NUM_CADASTRE")

# Identifier toutes les communes qui, en 2024, contiennent des codes de parcelles 
# avec des préfixes autres de "000" et les stocker dans une liste "Com_fusion" :
Cadastre_Geoloc_Bzh=Cadastre_Geoloc_Bzh[Cadastre_Geoloc_Bzh["millesime_Cadastre"]=="2024"]
Cadastre_Geoloc_Bzh["INSEE_COM"]=Cadastre_Geoloc_Bzh["NUM_CADASTRE"].str[:5]
Cadastre_Geoloc_Bzh["Prefixe"]=Cadastre_Geoloc_Bzh["NUM_CADASTRE"].str[5:8]
Com_Multi_prefixe=Cadastre_Geoloc_Bzh[Cadastre_Geoloc_Bzh["Prefixe"]!="000"]
Com_Fusion=Com_Multi_prefixe["INSEE_COM"].unique().tolist()

Cadastre_Geoloc_Bzh.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 7011876 entries, 0 to 7348307
Data columns (total 6 columns):
 #   Column              Dtype   
---  ------              -----   
 0   NUM_CADASTRE        object  
 1   millesime_Cadastre  object  
 2   contenance          float64 
 3   geometry            geometry
 4   INSEE_COM           object  
 5   Prefixe             object  
dtypes: float64(1), geometry(1), object(4)
memory usage: 374.5+ MB


In [11]:
# Les permis correctement géolocalisés, qui ne font pas partie des communes nouvelles issues de fusions :
SITADEL_Bzh_geoloc_OK=SITADEL_Bzh_geoloc_A[(SITADEL_Bzh_geoloc_A["geometry"].notna()) & (~SITADEL_Bzh_geoloc_A["INSEE_COM"].isin(Com_Fusion))]

SITADEL_Bzh_geoloc_OK["RESID_SECOND"]=SITADEL_Bzh_geoloc_OK["RESID_SECOND"].fillna("0")
SITADEL_Bzh_geoloc_OK["RESID_SECOND"]=SITADEL_Bzh_geoloc_OK["RESID_SECOND"].astype("int64")

SITADEL_Bzh_geoloc_OK["SIREN_EPCI"]=SITADEL_Bzh_geoloc_OK["SIREN_EPCI"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SURF_HABIT_NEW"]=SITADEL_Bzh_geoloc_OK["SURF_HABIT_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SURF_HOTEL_NEW"]=SITADEL_Bzh_geoloc_OK["SURF_HOTEL_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SURF_BUREAU_NEW"]=SITADEL_Bzh_geoloc_OK["SURF_BUREAU_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SURF_COMMERC_NEW"]=SITADEL_Bzh_geoloc_OK["SURF_COMMERC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SURF_ARTISAN_NEW"]=SITADEL_Bzh_geoloc_OK["SURF_ARTISAN_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SURF_INDUS_NEW"]=SITADEL_Bzh_geoloc_OK["SURF_INDUS_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SURF_ENTREPOT_NEW"]=SITADEL_Bzh_geoloc_OK["SURF_ENTREPOT_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SURF_SERV_PUB_NEW"]=SITADEL_Bzh_geoloc_OK["SURF_SERV_PUB_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["RESID_SECOND"]=SITADEL_Bzh_geoloc_OK["RESID_SECOND"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SIREN_DEMANDEUR"]=SITADEL_Bzh_geoloc_OK["SIREN_DEMANDEUR"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SIRET_DEMANDEUR"]=SITADEL_Bzh_geoloc_OK["SIRET_DEMANDEUR"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["TYPE_SERV_PUB"]=SITADEL_Bzh_geoloc_OK["TYPE_SERV_PUB"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["CAT_JURID_DEMANDEUR"]=SITADEL_Bzh_geoloc_OK["CAT_JURID_DEMANDEUR"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["NB_LOGEMENTS_NEW"]=SITADEL_Bzh_geoloc_OK["NB_LOGEMENTS_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["TYPE_HABIT_PRINCIP_SECOND"]=SITADEL_Bzh_geoloc_OK["TYPE_HABIT_PRINCIP_SECOND"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["SURF_AGRIC_NEW"]=SITADEL_Bzh_geoloc_OK["SURF_AGRIC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["NATURE_PROJET"]=SITADEL_Bzh_geoloc_OK["NATURE_PROJET"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["NATURE_PROJET_DETAIL"]=SITADEL_Bzh_geoloc_OK["NATURE_PROJET_DETAIL"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK["DEM_INSEE_COM"]=SITADEL_Bzh_geoloc_OK["DEM_INSEE_COM"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_OK.replace({"EPCI_DEMANDEUR": {"NR": "0"}}, inplace=True)
SITADEL_Bzh_geoloc_OK["EPCI_DEMANDEUR"]=SITADEL_Bzh_geoloc_OK["EPCI_DEMANDEUR"].fillna("0").astype("int64")


# Trouver le centroid des parcelles :
SITADEL_Bzh_geoloc_OK=gpd.GeoDataFrame(SITADEL_Bzh_geoloc_OK, geometry="geometry", crs="EPSG:4326")
SITADEL_Bzh_geoloc_OK["CENTROID"]=SITADEL_Bzh_geoloc_OK["geometry"].centroid
SITADEL_Bzh_geoloc_OK.drop(columns={"geometry"}, inplace=True)
SITADEL_Bzh_geoloc_OK=gpd.GeoDataFrame(SITADEL_Bzh_geoloc_OK, geometry="CENTROID", crs="EPSG:4326")
SITADEL_Bzh_geoloc_OK.rename(columns={"CENTROID":"geometry"}, inplace=True)
SITADEL_Bzh_geoloc_OK.info()

/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_96561/848777193.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SITADEL_Bzh_geoloc_OK["RESID_SECOND"]=SITADEL_Bzh_geoloc_OK["RESID_SECOND"].fillna("0")
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_96561/848777193.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SITADEL_Bzh_geoloc_OK["RESID_SECOND"]=SITADEL_Bzh_geoloc_OK["RESID_SECOND"].astype("int64")
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_96561/848777193.py:

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3959 entries, 0 to 4670
Data columns (total 57 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Type_AU                    3959 non-null   object        
 1   DEP                        3959 non-null   object        
 2   INSEE_COM                  3959 non-null   object        
 3   NOM_COMMUNE                3959 non-null   object        
 4   SIREN_EPCI                 3959 non-null   int64         
 5   NOM_EPCI                   3956 non-null   object        
 6   NUM_PERMIS                 3959 non-null   object        
 7   NUM_CADASTRE               3959 non-null   object        
 8   DATE                       3959 non-null   object        
 9   ANNEE_MOIS_AUTORISATION    3959 non-null   datetime64[ns]
 10  PERMIS_ANNULE              3959 non-null   object        
 11  SUPERF_TERRAIN             3959 non-null   object        
 12  DES

In [12]:
# Les permis qui n'ont pas pu être géolocalisés avec le cadastre (à traiter avec la BAN), qui font partie des communes nouvelles issues de fusions :
SITADEL_Bzh_geoloc_NUL=SITADEL_Bzh_geoloc_A[(SITADEL_Bzh_geoloc_A["geometry"].isna()) | (SITADEL_Bzh_geoloc_A["INSEE_COM"].isin(Com_Fusion))]
SITADEL_Bzh_geoloc_NUL=SITADEL_Bzh_geoloc_NUL[['Type_AU', 'DEP', 'INSEE_COM', 'NOM_COMMUNE',
       'SIREN_EPCI', 'NOM_EPCI', 'NUM_PERMIS', 'NUM_CADASTRE', 'DATE',
       'ANNEE_MOIS_AUTORISATION', 'PERMIS_ANNULE', 'SUPERF_TERRAIN',
       'DESTINATION', 'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW', 'NB_LOGEMENTS_NEW',
       'TYPE_HABIT_PRINCIP_SECOND', 'RESID_SECOND', 'CODE_ACTIV_DEMANDEUR',
       'CAT_JURID_DEMANDEUR', 'NOM_DEMANDEUR', 'SIREN_DEMANDEUR',
       'SIRET_DEMANDEUR', 'DEP_DEMANDEUR', 'DEM_INSEE_COM',
       'NOM_COMMUNE_DEMANDEUR', 'EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE', 'DEM_MEME_EPCI', 'DEM_MEME_DEP', 'DEM_BRETAGNE',
       'DEM_Ouest', 'DEM_LOINTAINS', 'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN',
       'NOM_VOIE_TERRAIN', 'CP_TERRAIN', 'sec_cadastre1', 'num_cadastre1',
       'sec_cadastre2', 'num_cadastre2', 'sec_cadastre3', 'num_cadastre3',
       'NATURE_PROJET', 'NATURE_PROJET_DETAIL', 'SURF_HOTEL_NEW',
       'SURF_BUREAU_NEW', 'SURF_COMMERC_NEW', 'SURF_ARTISAN_NEW',
       'SURF_INDUS_NEW', 'SURF_AGRIC_NEW', 'SURF_ENTREPOT_NEW',
       'SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB']]
SITADEL_Bzh_geoloc_NUL.to_csv("SITADEL_Bzh_geoloc_NUL.csv")

### Compléter le géocodage avec l'adresse de la parcelle, en interrogeant la BAN via l'API Adresse
# Dans le terminal - Doc : https://adresse.data.gouv.fr/api-doc/adresse*
# - curl -X POST -F data=@SITADEL_Bzh_geoloc_NUL.csv -F columns=NUM_VOIE_TERRAIN -F columns=TYPE_VOIE-TERRAIN -F columns=NOM_VOIE_TERRAIN columns=NOM_COMMUNE -F citycode=INSEE_COM https://api-adresse.data.gouv.fr/search/csv/ > SITADEL_Bzh_geoloc_NUL_GEOCOD.csv

In [13]:
# Chargement du résultat du géocodage avec l'API Adresse :
SITADEL_Bzh_geoloc_NUL_GEOCOD=pd.read_csv("SITADEL_Bzh_geoloc_NUL_GEOCOD.csv")
SITADEL_Bzh_geoloc_NUL_GEOCOD=gpd.GeoDataFrame(
    SITADEL_Bzh_geoloc_NUL_GEOCOD, geometry=gpd.points_from_xy(SITADEL_Bzh_geoloc_NUL_GEOCOD.longitude, SITADEL_Bzh_geoloc_NUL_GEOCOD.latitude), 
    crs="EPSG:4326")


SITADEL_Bzh_geoloc_NUL_GEOCOD["RESID_SECOND"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["RESID_SECOND"].fillna("0")
SITADEL_Bzh_geoloc_NUL_GEOCOD["RESID_SECOND"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["RESID_SECOND"].astype("int64")

SITADEL_Bzh_geoloc_NUL_GEOCOD["SIREN_EPCI"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SIREN_EPCI"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_HABIT_NEW"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_HABIT_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_HOTEL_NEW"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_HOTEL_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_BUREAU_NEW"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_BUREAU_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_COMMERC_NEW"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_COMMERC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_ARTISAN_NEW"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_ARTISAN_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_INDUS_NEW"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_INDUS_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_ENTREPOT_NEW"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_ENTREPOT_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_SERV_PUB_NEW"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_SERV_PUB_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SIREN_DEMANDEUR"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SIREN_DEMANDEUR"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SIRET_DEMANDEUR"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SIRET_DEMANDEUR"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["TYPE_SERV_PUB"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["TYPE_SERV_PUB"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["CAT_JURID_DEMANDEUR"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["CAT_JURID_DEMANDEUR"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["NB_LOGEMENTS_NEW"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["NB_LOGEMENTS_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["TYPE_HABIT_PRINCIP_SECOND"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["TYPE_HABIT_PRINCIP_SECOND"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_AGRIC_NEW"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["SURF_AGRIC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["NATURE_PROJET"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["NATURE_PROJET"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["NATURE_PROJET_DETAIL"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["NATURE_PROJET_DETAIL"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD["DEM_INSEE_COM"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["DEM_INSEE_COM"].fillna("0").astype("int64")
SITADEL_Bzh_geoloc_NUL_GEOCOD.loc[SITADEL_Bzh_geoloc_NUL_GEOCOD["EPCI_DEMANDEUR"].str.len()<8, "EPCI_DEMANDEUR"]="0"
SITADEL_Bzh_geoloc_NUL_GEOCOD["EPCI_DEMANDEUR"]=SITADEL_Bzh_geoloc_NUL_GEOCOD["EPCI_DEMANDEUR"].str[:8].fillna("0").astype(int)

# Attention aux résultats trop médiocres de géocodage
SITADEL_Bzh_geoloc_NUL_GEOCOD=SITADEL_Bzh_geoloc_NUL_GEOCOD[SITADEL_Bzh_geoloc_NUL_GEOCOD["result_score"]>=0.5]

SITADEL_Bzh_geoloc_NUL_GEOCOD.to_file("SITADEL_Bzh_geoloc_NUL_GEOCOD.geojson")
SITADEL_Bzh_geoloc_NUL_GEOCOD.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 556 entries, 0 to 711
Data columns (total 74 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Unnamed: 0                 556 non-null    int64   
 1   Type_AU                    556 non-null    object  
 2   DEP                        556 non-null    int64   
 3   INSEE_COM                  556 non-null    int64   
 4   NOM_COMMUNE                556 non-null    object  
 5   SIREN_EPCI                 556 non-null    int64   
 6   NOM_EPCI                   555 non-null    object  
 7   NUM_PERMIS                 556 non-null    object  
 8   NUM_CADASTRE               510 non-null    object  
 9   DATE                       556 non-null    object  
 10  ANNEE_MOIS_AUTORISATION    556 non-null    object  
 11  PERMIS_ANNULE              556 non-null    object  
 12  SUPERF_TERRAIN             556 non-null    int64   
 13  DESTINATION                556 n

##### Fusion des trois dataframes

In [23]:
# Fusion des trois dataframes :
Koumoul_Bzh_geo=gpd.read_file("SITADEL/Koumoul_BZH_2014_2024.geojson")
SITADEL_Bzh=pd.concat([Koumoul_Bzh_geo, SITADEL_Bzh_geoloc_OK, SITADEL_Bzh_geoloc_NUL_GEOCOD])
SITADEL_Bzh=SITADEL_Bzh.set_crs("EPSG:4326")
SITADEL_Bzh=SITADEL_Bzh[['Type_AU','NUM_PERMIS', 'DATE',
       'ANNEE_MOIS_AUTORISATION', 'PERMIS_ANNULE', 'SUPERF_TERRAIN',
       'DESTINATION', 'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW', 'NB_LOGEMENTS_NEW',
       'TYPE_HABIT_PRINCIP_SECOND', 'RESID_SECOND', 'CODE_ACTIV_DEMANDEUR',
       'CAT_JURID_DEMANDEUR', 'NOM_DEMANDEUR', 'SIREN_DEMANDEUR',
       'SIRET_DEMANDEUR', 'DEP_DEMANDEUR', 'DEM_INSEE_COM',
       'NOM_COMMUNE_DEMANDEUR', 'EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE', 'DEM_MEME_EPCI', 'DEM_MEME_DEP', 'DEM_BRETAGNE',
       'DEM_Ouest', 'DEM_LOINTAINS', 'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN',
       'NOM_VOIE_TERRAIN', 'CP_TERRAIN', 'SEC_CADASTRE1', 'NUM_CADASTRE1', 'SEC_CADASTRE2',
       'NUM_CADASTRE2', 'SEC_CADASTRE3', 'NUM_CADASTRE3',
       'latitude', 'longitude', 'parcel_confidence', 'parcelle',
       'NATURE_PROJET', 'NATURE_PROJET_DETAIL', 'SURF_HOTEL_NEW',
       'SURF_BUREAU_NEW', 'SURF_COMMERC_NEW', 'SURF_ARTISAN_NEW',
       'SURF_INDUS_NEW', 'SURF_AGRIC_NEW', 'SURF_ENTREPOT_NEW',
       'SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB', 'geometry']]

SITADEL_Bzh.to_file("SITADEL/SITADEL_BZH_2014_2024.geojson")

SITADEL_Bzh_plat=SITADEL_Bzh[['Type_AU', 'NUM_PERMIS', 'DATE', 'ANNEE_MOIS_AUTORISATION',
       'PERMIS_ANNULE', 'SUPERF_TERRAIN', 'DESTINATION', 'SURF_HABIT_NEW',
       'SURF_LOCAUX_NEW', 'NB_LOGEMENTS_NEW', 'TYPE_HABIT_PRINCIP_SECOND',
       'RESID_SECOND', 'CODE_ACTIV_DEMANDEUR', 'CAT_JURID_DEMANDEUR',
       'NOM_DEMANDEUR', 'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR', 'DEP_DEMANDEUR',
       'DEM_INSEE_COM', 'NOM_COMMUNE_DEMANDEUR', 'EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE', 'DEM_MEME_EPCI', 'DEM_MEME_DEP', 'DEM_BRETAGNE',
       'DEM_Ouest', 'DEM_LOINTAINS', 'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN',
       'NOM_VOIE_TERRAIN', 'CP_TERRAIN', 'SEC_CADASTRE1', 'NUM_CADASTRE1',
       'SEC_CADASTRE2', 'NUM_CADASTRE2', 'SEC_CADASTRE3', 'NUM_CADASTRE3',
       'latitude', 'longitude', 'parcel_confidence', 'parcelle',
       'NATURE_PROJET', 'NATURE_PROJET_DETAIL', 'SURF_HOTEL_NEW',
       'SURF_BUREAU_NEW', 'SURF_COMMERC_NEW', 'SURF_ARTISAN_NEW',
       'SURF_INDUS_NEW', 'SURF_AGRIC_NEW', 'SURF_ENTREPOT_NEW',
       'SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB']]
SITADEL_Bzh_plat.to_csv("SITADEL/SITADEL_BZH_2014_2024.csv")

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 223031 entries, 0 to 711
Data columns (total 53 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Type_AU                    223031 non-null  object        
 1   NUM_PERMIS                 223031 non-null  object        
 2   DATE                       223031 non-null  object        
 3   ANNEE_MOIS_AUTORISATION    223031 non-null  datetime64[ns]
 4   PERMIS_ANNULE              223031 non-null  object        
 5   SUPERF_TERRAIN             223031 non-null  object        
 6   DESTINATION                223031 non-null  object        
 7   SURF_HABIT_NEW             223031 non-null  int64         
 8   SURF_LOCAUX_NEW            223031 non-null  object        
 9   NB_LOGEMENTS_NEW           223031 non-null  int64         
 10  TYPE_HABIT_PRINCIP_SECOND  223031 non-null  int64         
 11  RESID_SECOND               223031 non-null  int64   

#### Croisement des permis avec la bande littorale, les zonages des PLU et la tâche artificialisée de 2014

In [56]:
# Chargement des permis :
SITADEL_Bzh=gpd.read_file("SITADEL/SITADEL_BZH_2014_2024.geojson")
# Chargement des polygones des bandes littorales :
BandeComLitt=gpd.read_file("BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH_4326.geojson")
BandeComLitt=BandeComLitt[BandeComLitt["BANDE_LITT"]!="5_10km"]
BandeComLitt["ID_LITT"]=BandeComLitt["INSEE_COM"]+"_"+BandeComLitt["BANDE_LITT"]
BandeComLitt=BandeComLitt[['BANDE_LITT', 'ID_LITT','INSEE_COM','SURFACE_M2', 'geometry']]
SITADEL_Bzh=gpd.sjoin(SITADEL_Bzh, BandeComLitt, how="left")

SITADEL_Bzh=SITADEL_Bzh.merge(Com_BZH, on="INSEE_COM", how="left")
SITADEL_Bzh["DEP"]=SITADEL_Bzh["INSEE_COM"].str[:2]

SITADEL_Bzh["DATE"]=SITADEL_Bzh["DATE"].astype("str")
SITADEL_Bzh["ANNEE_MOIS_AUTORISATION"]=SITADEL_Bzh["ANNEE_MOIS_AUTORISATION"].astype("str")
SITADEL_Bzh["ANNEE"]=SITADEL_Bzh["ANNEE_MOIS_AUTORISATION"].str[:4]


SITADEL_Bzh=SITADEL_Bzh[['Type_AU', 'DEP', 'INSEE_COM', 'NOM_COMMUNE',
       'SIREN_EPCI', 'NOM_EPCI', 'ID_LITT', 'BANDE_LITT','NUM_PERMIS', 'DATE',
       'ANNEE_MOIS_AUTORISATION', 'ANNEE', 'PERMIS_ANNULE', 'SUPERF_TERRAIN',
       'DESTINATION', 'SURF_HABIT_NEW', 'SURF_LOCAUX_NEW', 'NB_LOGEMENTS_NEW',
       'TYPE_HABIT_PRINCIP_SECOND', 'RESID_SECOND', 'CODE_ACTIV_DEMANDEUR',
       'CAT_JURID_DEMANDEUR', 'NOM_DEMANDEUR', 'SIREN_DEMANDEUR',
       'SIRET_DEMANDEUR', 'DEP_DEMANDEUR', 'DEM_INSEE_COM',
       'NOM_COMMUNE_DEMANDEUR', 'EPCI_DEMANDEUR',
       'DEM_MEME_COMMUNE', 'DEM_MEME_EPCI', 'DEM_MEME_DEP', 'DEM_BRETAGNE',
       'DEM_Ouest', 'DEM_LOINTAINS', 'NUM_VOIE_TERRAIN', 'TYPE_VOIE-TERRAIN',
       'NOM_VOIE_TERRAIN', 'CP_TERRAIN', 'SEC_CADASTRE1', 'NUM_CADASTRE1', 'SEC_CADASTRE2',
       'NUM_CADASTRE2', 'SEC_CADASTRE3', 'NUM_CADASTRE3','parcelle',
       'NATURE_PROJET', 'NATURE_PROJET_DETAIL', 'SURF_HOTEL_NEW',
       'SURF_BUREAU_NEW', 'SURF_COMMERC_NEW', 'SURF_ARTISAN_NEW',
       'SURF_INDUS_NEW', 'SURF_AGRIC_NEW', 'SURF_ENTREPOT_NEW',
       'SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB', 'geometry']]

######### filtrer les permis délivrés dans la bande des 5 km du littoral :
SITADEL_Bzh_Litt=SITADEL_Bzh[SITADEL_Bzh["BANDE_LITT"].notna()]

### Les permis de construire sont-ils attribués en dehors de la zone déjà urbanisée en 2014 ?

# Chargement de la tache artificialisée 2014 découpée par communes et bandes littorales :
TacheUrb2014=gpd.read_file("BD_TOPO/TACHE_ARTIF_5km_BZH_2014_BandeLitt_DEF.geojson")

TacheUrb2014["geometry"]=TacheUrb2014["geometry"].to_crs("epsg:4326")
TacheUrb2014["TacheArtif2014"]="TacheUrb"
TacheUrb2014=TacheUrb2014[["geometry","TacheArtif2014"]]

# Jointure spatiale des deux fichiers et localisation des permis par rapport à la tâche urbaine de 2014 :
SITADEL_Bzh_Litt=gpd.sjoin(SITADEL_Bzh_Litt, TacheUrb2014, how="left")
SITADEL_Bzh_Litt.loc[SITADEL_Bzh_Litt["TacheArtif2014"].isna(), "TacheUrb2014"]="extérieur"
SITADEL_Bzh_Litt.loc[SITADEL_Bzh_Litt["TacheArtif2014"]=="TacheUrb", "TacheUrb2014"]="intérieur"
SITADEL_Bzh_Litt.loc[SITADEL_Bzh_Litt["TacheUrb2014"]=="extérieur", "Hors_TacheUrb2014"]="1"
SITADEL_Bzh_Litt["Hors_TacheUrb2014"]=SITADEL_Bzh_Litt["Hors_TacheUrb2014"].fillna("0")
SITADEL_Bzh_Litt.drop(columns="index_right", inplace=True)

# Chargement de la tache artificialisée 2024 découpée par communes et bandes littorales :
TacheUrb2024=gpd.read_file("BD_TOPO/TACHE_ARTIF_5km_BZH_2014_2024_BandeLitt_DEF.geojson")
TacheUrb2024["geometry"]=TacheUrb2024["geometry"].to_crs("epsg:4326")
TacheUrb2024["TacheArtif2024"]="TacheUrb"
TacheUrb2024=TacheUrb2024[["geometry","TacheArtif2024"]]
# Jointure spatiale des deux fichiers :
SITADEL_Bzh_Litt=gpd.sjoin(SITADEL_Bzh_Litt, TacheUrb2024, how="left")
SITADEL_Bzh_Litt.loc[(SITADEL_Bzh_Litt["TacheArtif2024"].isna()) & (SITADEL_Bzh_Litt["Hors_TacheUrb2014"]=="1"), "Hors_TacheUrb2024"]="1"
SITADEL_Bzh_Litt["Hors_TacheUrb2024"]=SITADEL_Bzh_Litt["Hors_TacheUrb2024"].fillna("0")
SITADEL_Bzh_Litt.drop(columns="index_right", inplace=True)

### Dans quel secteur des Plans locaux d'urbanisme (actuellement en vigueur) chaque permis de construire est-il situé ?
# Chargement des secteurs de documents d'urbanisme :
ZonesUrba=gpd.read_file("Zones_Urba_ComLitt_Bzh_new.shp")[["ZONE","libelle","libelong","SOURCE","geometry"]]
ZonesUrba.rename(columns={"ZONE":"ZoneUrba","libelle":"CodeZoneUrba","libelong":"NomZoneUrba", "SOURCE":"SourceZoneUrba"}, inplace=True)
# Jointure spatiale des deux fichiers :
SITADEL_Bzh_Litt=gpd.sjoin(SITADEL_Bzh_Litt, ZonesUrba, how="left")

SITADEL_Bzh_Litt["Latitude"] = SITADEL_Bzh_Litt["geometry"].y
SITADEL_Bzh_Litt["Longitude"] = SITADEL_Bzh_Litt["geometry"].x

SITADEL_Bzh_Litt=SITADEL_Bzh_Litt[['Type_AU', 'DEP', 'INSEE_COM','NOM_COMMUNE','SIREN_EPCI', 'NOM_EPCI', 'ID_LITT','BANDE_LITT', 
    'TacheUrb2014','Hors_TacheUrb2014','Hors_TacheUrb2024','ZoneUrba', 
    'CodeZoneUrba','NomZoneUrba','SourceZoneUrba','NUM_PERMIS', 'DATE', 'ANNEE_MOIS_AUTORISATION', 'ANNEE',
    'PERMIS_ANNULE', 'SUPERF_TERRAIN', 'DESTINATION', 'SURF_HABIT_NEW','SURF_LOCAUX_NEW', 'NB_LOGEMENTS_NEW', 
    'TYPE_HABIT_PRINCIP_SECOND','RESID_SECOND', 'CODE_ACTIV_DEMANDEUR', 'CAT_JURID_DEMANDEUR','NOM_DEMANDEUR', 
    'SIREN_DEMANDEUR', 'SIRET_DEMANDEUR', 'DEP_DEMANDEUR','DEM_INSEE_COM', 'NOM_COMMUNE_DEMANDEUR',
    'EPCI_DEMANDEUR', 'DEM_MEME_COMMUNE', 'DEM_MEME_EPCI', 'DEM_MEME_DEP','DEM_BRETAGNE', 'DEM_Ouest', 'DEM_LOINTAINS',
    'parcelle', 'NATURE_PROJET', 'NATURE_PROJET_DETAIL','SURF_HOTEL_NEW', 'SURF_BUREAU_NEW', 'SURF_COMMERC_NEW','SURF_ARTISAN_NEW', 
    'SURF_INDUS_NEW', 'SURF_AGRIC_NEW','SURF_ENTREPOT_NEW', 'SURF_SERV_PUB_NEW', 'TYPE_SERV_PUB', 'Latitude','Longitude', 'geometry']]

SITADEL_Bzh_Litt.to_file("SITADEL/SITADEL_BZH_2014_2024_LITT_ALL.geojson")
SITADEL_Bzh_Litt.to_csv("SITADEL/SITADEL_BZH_2014_2024_LITT_ALL.csv")

SITADEL_Bzh_Litt.info()

/Users/denisvannier/opt/anaconda3/envs/GEODATA2/lib/python3.9/site-packages/pyogrio/raw.py:196: RuntimeWarning: Zones_Urba_ComLitt_Bzh_new.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 77824 entries, 560 to 223030
Data columns (total 57 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Type_AU                    77824 non-null  object  
 1   DEP                        77824 non-null  object  
 2   INSEE_COM                  77824 non-null  object  
 3   NOM_COMMUNE                77824 non-null  object  
 4   SIREN_EPCI                 77824 non-null  object  
 5   NOM_EPCI                   77771 non-null  object  
 6   ID_LITT                    77824 non-null  object  
 7   BANDE_LITT                 77824 non-null  object  
 8   TacheUrb2014               77824 non-null  object  
 9   Hors_TacheUrb2014          77824 non-null  object  
 10  Hors_TacheUrb2024          77824 non-null  object  
 11  ZoneUrba                   58969 non-null  object  
 12  CodeZoneUrba               58969 non-null  object  
 13  NomZoneUrba              

#### Permis 2014-2024 NEW

In [57]:
########## Les permis responsables de l'extension des zones artificialisées entre 2014 et 2024 :
# Filtrage et export des permis délivrés pour des parcelles situées uniquement dans les secteurs artificialisés entre 2014 et 2024 :
SITADEL_Bzh_Litt_New2014=SITADEL_Bzh_Litt[SITADEL_Bzh_Litt["Hors_TacheUrb2014"]=="1"]
SITADEL_Bzh_Litt_New2014=SITADEL_Bzh_Litt_New2014[SITADEL_Bzh_Litt_New2014["Hors_TacheUrb2024"]=="0"]

SITADEL_Bzh_Litt_New2014=SITADEL_Bzh_Litt_New2014[SITADEL_Bzh_Litt_New2014["PERMIS_ANNULE"]=="non"]
SSITADEL_Bzh_Litt_New2014=SITADEL_Bzh_Litt_New2014.reset_index()
SITADEL_Bzh_Litt_New2014.rename(columns={"index":"ID_PERMIS"}, inplace=True)

SITADEL_Bzh_Litt_New2014.to_file("SITADEL/SITADEL_BZH_2014_2024_LITT_NEW2014.geojson")
SITADEL_Bzh_Litt_New2014.to_csv("SITADEL/SITADEL_BZH_2014_2024_LITT_NEW2014.csv")
SITADEL_Bzh_Litt_New2014.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 18539 entries, 1404 to 222853
Data columns (total 57 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Type_AU                    18539 non-null  object  
 1   DEP                        18539 non-null  object  
 2   INSEE_COM                  18539 non-null  object  
 3   NOM_COMMUNE                18539 non-null  object  
 4   SIREN_EPCI                 18539 non-null  object  
 5   NOM_EPCI                   18538 non-null  object  
 6   ID_LITT                    18539 non-null  object  
 7   BANDE_LITT                 18539 non-null  object  
 8   TacheUrb2014               18539 non-null  object  
 9   Hors_TacheUrb2014          18539 non-null  object  
 10  Hors_TacheUrb2024          18539 non-null  object  
 11  ZoneUrba                   14585 non-null  object  
 12  CodeZoneUrba               14585 non-null  object  
 13  NomZoneUrba             

#### Permis 2020-2024

In [58]:
########## Les permis qui annoncent l'artificialisation à venir :
# Filtrage et export des permis délivrés depuis le 1er janvier 2020 (et donc potentiellement valides jusqu'en janvier 2025), 
# en dehors du secteurs déjà artificialisé en 2024 :
SITADEL_Bzh_New2024=SITADEL_Bzh_Litt[SITADEL_Bzh_Litt["Hors_TacheUrb2024"]=="1"]
SITADEL_Bzh_New2024=SITADEL_Bzh_New2024[SITADEL_Bzh_New2024["PERMIS_ANNULE"]=="non"]
SITADEL_Bzh_New2024=SITADEL_Bzh_New2024[SITADEL_Bzh_New2024["ANNEE_MOIS_AUTORISATION"]>="2020-01-01 01:00:00"]

SITADEL_Bzh_New2024=SITADEL_Bzh_New2024.reset_index()
SITADEL_Bzh_New2024.rename(columns={"index":"ID_PERMIS"}, inplace=True)

SITADEL_Bzh_New2024.to_file("SITADEL/SITADEL_BZH_2020_2024_LITT_NEW2024.geojson")
SITADEL_Bzh_New2024.to_csv("SITADEL/SITADEL_BZH_2020_2024_LITT_NEW2024.csv")
SITADEL_Bzh_New2024.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7404 entries, 0 to 7403
Data columns (total 58 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   ID_PERMIS                  7404 non-null   int64   
 1   Type_AU                    7404 non-null   object  
 2   DEP                        7404 non-null   object  
 3   INSEE_COM                  7404 non-null   object  
 4   NOM_COMMUNE                7404 non-null   object  
 5   SIREN_EPCI                 7404 non-null   object  
 6   NOM_EPCI                   7400 non-null   object  
 7   ID_LITT                    7404 non-null   object  
 8   BANDE_LITT                 7404 non-null   object  
 9   TacheUrb2014               7404 non-null   object  
 10  Hors_TacheUrb2014          7404 non-null   object  
 11  Hors_TacheUrb2024          7404 non-null   object  
 12  ZoneUrba                   5812 non-null   object  
 13  CodeZoneUrba             

#### Permis 2020-2024 CARTO

In [59]:
##### Version adaptée à la carte des permis délivrés depuis janvier 2020 en dehors des zones déjà artificialisées en 2024, avec des textes pour les popups :

SITADEL_Bzh_New2024=gpd.read_file("SITADEL/SITADEL_BZH_2020_2024_LITT_NEW2024.geojson")

SITADEL_Carto=SITADEL_Bzh_New2024

# Transformation de la date :
SITADEL_Carto["DATE"]=pd.to_datetime(SITADEL_Carto["DATE"])
SITADEL_Carto["jour"]=SITADEL_Carto["DATE"].dt.day
SITADEL_Carto["mois"]=SITADEL_Carto["DATE"].dt.month
SITADEL_Carto["annee"]=SITADEL_Carto["DATE"].dt.year

SITADEL_Carto["jour"]=SITADEL_Carto["jour"].astype("str")
SITADEL_Carto["mois"]=SITADEL_Carto["mois"].astype("str")
SITADEL_Carto["annee"]=SITADEL_Carto["annee"].astype("str")

SITADEL_Carto.replace({"mois": {"1": "janvier", "2":"février", "3": "mars", "4":"avril", "5":"mai", "6":"juin", "7":"juillet", "8":"août", "9":"septembre", "10":"octobre", "11":"novembre", "12":"décembre"}}, inplace=True)

SITADEL_Carto["DATE_new"]=SITADEL_Carto["jour"]+" "+SITADEL_Carto["mois"]+" "+SITADEL_Carto["annee"]

# Dans le cas d'un permis pour un local : 
SITADEL_Carto.loc[SITADEL_Carto["SURF_HOTEL_NEW"]>=1, "TypeLocal"]="hôtel"
SITADEL_Carto.loc[SITADEL_Carto["SURF_BUREAU_NEW"]>=1, "TypeLocal"]="bureaux"
SITADEL_Carto.loc[SITADEL_Carto["SURF_ARTISAN_NEW"]>=1, "TypeLocal"]="artisanat"
SITADEL_Carto.loc[SITADEL_Carto["SURF_COMMERC_NEW"]>=1, "TypeLocal"]="commerce"
SITADEL_Carto.loc[SITADEL_Carto["SURF_INDUS_NEW"]>=1, "TypeLocal"]="industrie"
SITADEL_Carto.loc[SITADEL_Carto["SURF_ENTREPOT_NEW"]>=1, "TypeLocal"]="entrepôt"
SITADEL_Carto.loc[SITADEL_Carto["SURF_AGRIC_NEW"]>=1, "TypeLocal"]="bâtiment agricole"
SITADEL_Carto.loc[SITADEL_Carto["SURF_SERV_PUB_NEW"]>=1, "TypeLocal"]="service public"

SITADEL_Carto["DEP"]=SITADEL_Carto["DEP"].astype("str")
SITADEL_Carto["SURF_LOCAUX_NEW"]=SITADEL_Carto["SURF_LOCAUX_NEW"].astype("str")
SITADEL_Carto["SURF_HABIT_NEW"]=SITADEL_Carto["SURF_HABIT_NEW"].astype("str")
SITADEL_Carto["NB_LOGEMENTS_NEW"]=SITADEL_Carto["NB_LOGEMENTS_NEW"].astype("str")

SITADEL_Carto.loc[(SITADEL_Carto["Type_AU"]=="Locaux") & (SITADEL_Carto["NOM_DEMANDEUR"].notna()), "TEXTE_POPUP"]="Permis accordé le " + SITADEL_Carto["DATE_new"] + " par la commune de " + SITADEL_Carto["NOM_COMMUNE"] + " ("+SITADEL_Carto["DEP"] + ") à " + SITADEL_Carto["NOM_DEMANDEUR"] + " pour la construction de " + SITADEL_Carto["SURF_LOCAUX_NEW"] + " m2 de locaux professionnels (" + SITADEL_Carto["TypeLocal"] + ")."
SITADEL_Carto.loc[(SITADEL_Carto["Type_AU"]=="Locaux") & (SITADEL_Carto["NOM_DEMANDEUR"].isna()), "TEXTE_POPUP"]="Permis accordé le " + SITADEL_Carto["DATE_new"] + " par la commune de " + SITADEL_Carto["NOM_COMMUNE"] + " ("+SITADEL_Carto["DEP"] + ") pour la construction de " + SITADEL_Carto["SURF_LOCAUX_NEW"] + " m2 de locaux professionnels (" + SITADEL_Carto["TypeLocal"] + ")."

# Dans le cas d'un permis pour une maison individuelle :
SITADEL_Carto.loc[(SITADEL_Carto["Type_AU"]=="Logements") & (SITADEL_Carto["NB_LOGEMENTS_NEW"]=="1"), "TEXTE_POPUP"]="Permis accordé le "+SITADEL_Carto["DATE_new"]+" par la commune de "+SITADEL_Carto["NOM_COMMUNE"]+" ("+SITADEL_Carto["DEP"]+") pour la construction d'une habitation individuelle de "+ SITADEL_Carto["SURF_HABIT_NEW"]+"m2."

# Dans le cas d'un permis pour une résidence secondaire :
SITADEL_Carto.loc[(SITADEL_Carto["Type_AU"]=="Logements") & (SITADEL_Carto["NB_LOGEMENTS_NEW"]=="1") & (SITADEL_Carto["RESID_SECOND"]==1), "TEXTE_POPUP"]="Permis accordé le "+SITADEL_Carto["DATE_new"]+" par la commune de "+SITADEL_Carto["NOM_COMMUNE"]+" ("+SITADEL_Carto["DEP"]+") pour la construction d'une résidence secondaire de "+ SITADEL_Carto["SURF_HABIT_NEW"]+"m2."

# Dans d'un permis pour un groupe de logements :
SITADEL_Carto.loc[(SITADEL_Carto["Type_AU"]=="Logements") & (SITADEL_Carto["NB_LOGEMENTS_NEW"]!="1") & (SITADEL_Carto["NB_LOGEMENTS_NEW"]!="0"), "TEXTE_POPUP"] = "Permis accordé le "+SITADEL_Carto["DATE_new"]+" par la commune de "+SITADEL_Carto["NOM_COMMUNE"]+" ("+SITADEL_Carto["DEP"]+") pour la construction de "+ SITADEL_Carto["NB_LOGEMENTS_NEW"]+" logements ("+SITADEL_Carto["SURF_HABIT_NEW"]+"m2 de surface habitable."

# Dans le cas d'un permis délivré dans une zone protégée (naturelle ou agricole) :
SITADEL_Carto.loc[SITADEL_Carto["ZoneUrba"]=="zone protégée (naturelle ou agricole)", "TEXTE_POPUP"]=SITADEL_Carto["TEXTE_POPUP"] + " La parcelle est située dans une zone protégée (naturelle ou agricole) selon le document d'urbanisme actuellement en vigueur."

SITADEL_Carto_TEXT=SITADEL_Carto[['ID_PERMIS', 'DEP', 'INSEE_COM', 'NOM_COMMUNE', 'SIREN_EPCI','NOM_EPCI', 'ID_LITT', 'BANDE_LITT','ANNEE_MOIS_AUTORISATION','TEXTE_POPUP','geometry']]
SITADEL_Carto_TEXT.to_file("SITADEL/SITADEL_Bzh_Litt_2024New_Carto.geojson")
SITADEL_Carto_TEXT.info()
SITADEL_Carto_TEXT.sample(3)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7404 entries, 0 to 7403
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ID_PERMIS                7404 non-null   int32         
 1   DEP                      7404 non-null   object        
 2   INSEE_COM                7404 non-null   object        
 3   NOM_COMMUNE              7404 non-null   object        
 4   SIREN_EPCI               7404 non-null   object        
 5   NOM_EPCI                 7400 non-null   object        
 6   ID_LITT                  7404 non-null   object        
 7   BANDE_LITT               7404 non-null   object        
 8   ANNEE_MOIS_AUTORISATION  7404 non-null   datetime64[ms]
 9   TEXTE_POPUP              7404 non-null   object        
 10  geometry                 7404 non-null   geometry      
dtypes: datetime64[ms](1), geometry(1), int32(1), object(8)
memory usage: 607.5+ KB


,ID_PERMIS,DEP,INSEE_COM,NOM_COMMUNE,SIREN_EPCI,NOM_EPCI,ID_LITT,BANDE_LITT,ANNEE_MOIS_AUTORISATION,TEXTE_POPUP,geometry
7155,220255,35,35284,Saint-Jouan-des-Guérets,243500782,CA du Pays de Saint Malo Agglomération,35284_500m_1km,500m_1km,2024-09-01 01:00:00,Permis accordé le 3 septembre 2024 par la comm...,POINT (-1.98657 48.60239)
1912,32250,29,29057,La Forêt-Fouesnant,242900660,CC du Pays Fouesnantais,29057_100_200m,100_200m,2023-01-01 01:00:00,Permis accordé le 20 janvier 2023 par la commu...,POINT (-3.96719 47.89747)
2943,46246,29,29184,Plouénan,200067072,CC Haut-Léon Communauté,29184_1_5km,1_5km,2021-07-01 01:00:00,Permis accordé le 21 juillet 2021 par la commu...,POINT (-3.985 48.62573)


## SYNTHESES

### Synthèse des tous permis délivrés depuis 2013 à moins de 5km de la côte

In [7]:
SITADEL_Bzh_Litt=pd.read_csv("SITADEL/SITADEL_BZH_2014_2024_LITT_ALL.csv")

# Préparer les variables pour les synthèses :
SITADEL_Bzh_Litt.replace({"PERMIS_ANNULE": {"non": "0", "oui": "1"}}, inplace=True)
SITADEL_Bzh_Litt["PERMIS_ANNULE"]=SITADEL_Bzh_Litt["PERMIS_ANNULE"].astype("int64")
SITADEL_Bzh_Litt.replace({"RESID_SECOND": {"0.0": "0", "":"0", "1.0": "1"}}, inplace=True)

SITADEL_Bzh_Litt["Hors_TacheUrb2014"]=SITADEL_Bzh_Litt["Hors_TacheUrb2014"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["Hors_TacheUrb2024"]=SITADEL_Bzh_Litt["Hors_TacheUrb2024"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["RESID_SECOND"]=SITADEL_Bzh_Litt["RESID_SECOND"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SUPERF_TERRAIN"]=SITADEL_Bzh_Litt["SUPERF_TERRAIN"].astype("float64")
SITADEL_Bzh_Litt["SURF_HABIT_NEW"]=SITADEL_Bzh_Litt["SURF_HABIT_NEW"].astype("float64")
SITADEL_Bzh_Litt["SURF_LOCAUX_NEW"]=SITADEL_Bzh_Litt["SURF_LOCAUX_NEW"].astype("float64")
SITADEL_Bzh_Litt["SURF_HOTEL_NEW"]=SITADEL_Bzh_Litt["SURF_HOTEL_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_BUREAU_NEW"]=SITADEL_Bzh_Litt["SURF_BUREAU_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_COMMERC_NEW"]=SITADEL_Bzh_Litt["SURF_COMMERC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_ARTISAN_NEW"]=SITADEL_Bzh_Litt["SURF_ARTISAN_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_INDUS_NEW"]=SITADEL_Bzh_Litt["SURF_INDUS_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_AGRIC_NEW"]=SITADEL_Bzh_Litt["SURF_AGRIC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_ENTREPOT_NEW"]=SITADEL_Bzh_Litt["SURF_ENTREPOT_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_SERV_PUB_NEW"]=SITADEL_Bzh_Litt["SURF_SERV_PUB_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_MEME_COMMUNE"]=SITADEL_Bzh_Litt["DEM_MEME_COMMUNE"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_MEME_EPCI"]=SITADEL_Bzh_Litt["DEM_MEME_EPCI"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_MEME_DEP"]=SITADEL_Bzh_Litt["DEM_MEME_DEP"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_BRETAGNE"]=SITADEL_Bzh_Litt["DEM_BRETAGNE"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_Ouest"]=SITADEL_Bzh_Litt["DEM_Ouest"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_LOINTAINS"]=SITADEL_Bzh_Litt["DEM_LOINTAINS"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["NB_LOGEMENTS_NEW"]=SITADEL_Bzh_Litt["NB_LOGEMENTS_NEW"].fillna("0").astype("int64")

SITADEL_Bzh_Litt.info()


/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_79125/3938599693.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  SITADEL_Bzh_Litt=pd.read_csv("SITADEL/SITADEL_BZH_2014_2024_LITT_ALL.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77824 entries, 0 to 77823
Data columns (total 58 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 77824 non-null  int64  
 1   Type_AU                    77824 non-null  object 
 2   DEP                        77824 non-null  int64  
 3   INSEE_COM                  77824 non-null  int64  
 4   NOM_COMMUNE                77824 non-null  object 
 5   SIREN_EPCI                 77824 non-null  object 
 6   NOM_EPCI                   77771 non-null  object 
 7   ID_LITT                    77824 non-null  object 
 8   BANDE_LITT                 77824 non-null  object 
 9   TacheUrb2014               77824 non-null  object 
 10  Hors_TacheUrb2014          77824 non-null  int64  
 11  Hors_TacheUrb2024          77824 non-null  int64  
 12  ZoneUrba                   58969 non-null  object 
 13  CodeZoneUrba               58938 non-null  obj

In [8]:
#### Synthèse par communes
SITADEL_Litt_Bilan=SITADEL_Bzh_Litt.groupby(["INSEE_COM","NOM_COMMUNE","SIREN_EPCI","TacheUrb2014"], as_index=False).agg(
   Nb_Permis=("INSEE_COM","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"),
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum")
   )

SITADEL_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Litt_Bilan["Permis_annules"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_MEME_COMMUN"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_COMMUNE"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Litt_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Litt_Bilan=SITADEL_Litt_Bilan[['INSEE_COM','NOM_COMMUNE','SIREN_EPCI', 'TacheUrb2014', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_COMMUN', 'Part_Demandeurs_MEME_EPCI',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Litt_Bilan.to_csv("SITADEL/SITADEL_BZH_2014_2024_LITT_ALL_Bilan_COM.csv")
SITADEL_Litt_Bilan.sample(5)

,INSEE_COM,NOM_COMMUNE,SIREN_EPCI,TacheUrb2014,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_COMMUN,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
52,22168,Perros-Guirec,200065928,intérieur,302,6.000000000,25.000000000,56.000000000,71.000000000,80.000000000,81.000000000,19.000000000,928,40,543114.000000000,101257.000000000,29444.000000000,12770,364,5201,384,631,0,2264,4431
455,29272,Saint-Yvi,242900769,extérieur,6,0.000000000,67.000000000,83.000000000,83.000000000,83.000000000,83.000000000,17.000000000,1,0,81658.000000000,124.000000000,2579.000000000,0,0,0,0,0,2579,0,0
567,35288,Saint-Malo,243500782,intérieur,603,6.000000000,40.000000000,47.000000000,89.000000000,95.000000000,96.000000000,4.000000000,5726,65,891189.000000000,405393.000000000,39382.000000000,0,0,0,0,0,0,0,0
11,22054,Erquy,200069391,extérieur,158,1.000000000,25.000000000,44.000000000,61.000000000,73.000000000,82.000000000,18.000000000,186,22,607287.000000000,22272.000000000,17500.000000000,218,0,0,3073,177,5343,0,4421
42,22127,Lézardrieux,200065928,intérieur,49,6.000000000,45.000000000,63.000000000,92.000000000,96.000000000,96.000000000,4.000000000,53,1,233131.000000000,5362.000000000,16695.000000000,0,0,6,113,0,15940,0,636


### Synthèse des tous permis délivrés depuis 2013 en dehors des secteurs artificialisés en 2014

In [14]:
SITADEL_Bzh_Litt=gpd.read_file("SITADEL/SITADEL_BZH_2014_2024_LITT_NEW2014.csv")
SITADEL_Bzh_New2024=pd.read_csv("SITADEL/SITADEL_BZH_2020_2024_LITT_NEW2024.csv")
SITADEL_Bzh_Litt=pd.concat([SITADEL_Bzh_Litt, SITADEL_Bzh_New2024], ignore_index=True)
SITADEL_Bzh_Litt.drop(columns={"field_1","ID_PERMIS", "Unnamed: 0", "geometry"}, inplace=True)

# Préparer les variables pour les synthèses 
SITADEL_Bzh_Litt.replace({"PERMIS_ANNULE": {"non": "0", "oui": "1"}}, inplace=True)
SITADEL_Bzh_Litt["PERMIS_ANNULE"]=SITADEL_Bzh_Litt["PERMIS_ANNULE"].astype("int64")
SITADEL_Bzh_Litt.replace({"RESID_SECOND": {"0.0": "0", "":"0", "1.0": "1"}}, inplace=True)

SITADEL_Bzh_Litt["Hors_TacheUrb2014"]=SITADEL_Bzh_Litt["Hors_TacheUrb2014"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["Hors_TacheUrb2024"]=SITADEL_Bzh_Litt["Hors_TacheUrb2024"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["RESID_SECOND"]=SITADEL_Bzh_Litt["RESID_SECOND"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SUPERF_TERRAIN"]=SITADEL_Bzh_Litt["SUPERF_TERRAIN"].astype("float64")
SITADEL_Bzh_Litt["SURF_HABIT_NEW"]=SITADEL_Bzh_Litt["SURF_HABIT_NEW"].astype("float64")
SITADEL_Bzh_Litt["SURF_LOCAUX_NEW"]=SITADEL_Bzh_Litt["SURF_LOCAUX_NEW"].astype("float64")
SITADEL_Bzh_Litt["SURF_HOTEL_NEW"]=SITADEL_Bzh_Litt["SURF_HOTEL_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_BUREAU_NEW"]=SITADEL_Bzh_Litt["SURF_BUREAU_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_COMMERC_NEW"]=SITADEL_Bzh_Litt["SURF_COMMERC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_ARTISAN_NEW"]=SITADEL_Bzh_Litt["SURF_ARTISAN_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_INDUS_NEW"]=SITADEL_Bzh_Litt["SURF_INDUS_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_AGRIC_NEW"]=SITADEL_Bzh_Litt["SURF_AGRIC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_ENTREPOT_NEW"]=SITADEL_Bzh_Litt["SURF_ENTREPOT_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["SURF_SERV_PUB_NEW"]=SITADEL_Bzh_Litt["SURF_SERV_PUB_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_MEME_COMMUNE"]=SITADEL_Bzh_Litt["DEM_MEME_COMMUNE"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_MEME_EPCI"]=SITADEL_Bzh_Litt["DEM_MEME_EPCI"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_MEME_DEP"]=SITADEL_Bzh_Litt["DEM_MEME_DEP"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_BRETAGNE"]=SITADEL_Bzh_Litt["DEM_BRETAGNE"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_Ouest"]=SITADEL_Bzh_Litt["DEM_Ouest"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["DEM_LOINTAINS"]=SITADEL_Bzh_Litt["DEM_LOINTAINS"].fillna("0").astype("int64")
SITADEL_Bzh_Litt["NB_LOGEMENTS_NEW"]=SITADEL_Bzh_Litt["NB_LOGEMENTS_NEW"].fillna("0").astype("int64")

SITADEL_Bzh_Litt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25943 entries, 0 to 25942
Data columns (total 56 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Type_AU                    25943 non-null  object 
 1   DEP                        25943 non-null  object 
 2   INSEE_COM                  25943 non-null  object 
 3   NOM_COMMUNE                25943 non-null  object 
 4   SIREN_EPCI                 25943 non-null  object 
 5   NOM_EPCI                   25939 non-null  object 
 6   ID_LITT                    25943 non-null  object 
 7   BANDE_LITT                 25943 non-null  object 
 8   TacheUrb2014               25943 non-null  object 
 9   Hors_TacheUrb2014          25943 non-null  int64  
 10  Hors_TacheUrb2024          25943 non-null  int64  
 11  ZoneUrba                   24351 non-null  object 
 12  CodeZoneUrba               24347 non-null  object 
 13  NomZoneUrba                23866 non-null  obj

In [15]:
#### Synthèse par communes
SITADEL_Litt_Bilan=SITADEL_Bzh_Litt.groupby(["INSEE_COM","NOM_COMMUNE","SIREN_EPCI"], as_index=False).agg(
   Nb_Permis=("INSEE_COM","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"),
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum")
   )

SITADEL_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Litt_Bilan["Permis_annules"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_MEME_COMMUN"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_COMMUNE"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Litt_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Litt_Bilan=SITADEL_Litt_Bilan[['INSEE_COM','NOM_COMMUNE','SIREN_EPCI', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_COMMUN', 'Part_Demandeurs_MEME_EPCI',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Litt_Bilan.to_csv("SITADEL/SITADEL_BZH_2014_2024_LITT_NEW2014_Bilan_COM.csv")
SITADEL_Litt_Bilan.sample(5)

,INSEE_COM,NOM_COMMUNE,SIREN_EPCI,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_COMMUN,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
384,22179,Fréhel,200068989,60,0.000000000,5.000000000,10.000000000,80.000000000,85.000000000,88.000000000,12.000000000,57,7,53570.000000000,5953.000000000,1260.000000000,0,0,1260,0,0,0,0,0
59,22319,Saint-Michel-en-Grève,200065928,1,0.000000000,100.000000000,100.000000000,100.000000000,100.000000000,100.000000000,0.000000000,0,0,28981.000000000,0.000000000,163.000000000,0,0,0,0,0,163,0,0
647,56036,Caudan,200042174,39,0.000000000,26.000000000,74.000000000,97.000000000,100.000000000,100.000000000,0.000000000,81,0,49129.000000000,9653.000000000,13968.000000000,0,441,5184,5184,0,0,2808,117
687,56185,Quéven,200042174,152,0.000000000,26.000000000,84.000000000,93.000000000,98.000000000,98.000000000,2.000000000,248,1,586490.000000000,23805.000000000,3866.000000000,0,0,146,1626,0,0,1738,210
142,29155,Ouessant,NR,4,0.000000000,25.000000000,0.000000000,50.000000000,50.000000000,50.000000000,50.000000000,1,1,38464.000000000,94.000000000,335.000000000,0,0,189,0,0,0,50,96


In [16]:
#### Synthèse par EPCI
SITADEL_Litt_Bilan=SITADEL_Bzh_Litt.groupby(["SIREN_EPCI","NOM_EPCI"], as_index=False).agg(
   Nb_Permis=("SIREN_EPCI","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"),
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum")
   )

SITADEL_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Litt_Bilan["Permis_annules"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Litt_Bilan=SITADEL_Litt_Bilan[['SIREN_EPCI','NOM_EPCI', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_EPCI','Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Litt_Bilan.to_csv("SITADEL/SITADEL_BZH_2014_2024_LITT_NEW2014_Bilan_EPCI.csv")
SITADEL_Litt_Bilan.sample(5)

,SIREN_EPCI,NOM_EPCI,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
38,245614383,CC Questembert Communauté,5,0.000000000,40.000000000,100.000000000,100.000000000,100.000000000,0.000000000,4,0,30251.000000000,592.000000000,24.000000000,0,0,0,0,0,24,0,0
29,242900801,CA du Pays de Landerneau-Daoulas,969,0.000000000,46.000000000,94.000000000,96.000000000,96.000000000,4.000000000,1187,17,1269694.000000000,125590.000000000,44651.000000000,1595,4681,6162,2742,8097,13671,6019,1067
39,NR,,1,0.000000000,0.000000000,100.000000000,100.000000000,100.000000000,0.000000000,1,0,649.000000000,147.000000000,0.000000000,0,0,0,0,0,0,0,0
1,200042174,CA Lorient Agglomération,1085,0.000000000,74.000000000,85.000000000,93.000000000,94.000000000,6.000000000,1914,37,4073222.000000000,186530.000000000,175892.000000000,10443,26731,16601,20569,23100,8148,32409,26344
30,242900835,CA Morlaix Communauté,375,0.000000000,67.000000000,84.000000000,90.000000000,91.000000000,9.000000000,431,24,623539.000000000,47343.000000000,29225.000000000,0,2378,730,970,10809,10141,2104,2093


In [17]:
#### Synthèse par bande littorale
SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt.groupby(["BANDE_LITT"], as_index=False).agg(
   Nb_Permis=("Type_AU","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"),
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum"))

SITADEL_Bzh_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Bzh_Litt_Bilan["Permis_annules"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_COMMUN"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_COMMUNE"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt_Bilan[['BANDE_LITT', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_COMMUN', 'Part_Demandeurs_MEME_EPCI',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Bzh_Litt_Bilan.to_csv("SITADEL/SITADEL_BZH_2014_2024_LITT_NEW2014_Bilan_BandeLitt.csv")
SITADEL_Bzh_Litt_Bilan.sample(5)

,BANDE_LITT,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_COMMUN,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
4,500m_1km,5813,0.000000000,23.000000000,48.000000000,76.000000000,86.000000000,89.000000000,11.000000000,7950,539,12713961.000000000,805837.000000000,208297.000000000,7128,25212,20220,24470,6245,64252,34797,19354
2,1_5km,16976,0.000000000,26.000000000,53.000000000,82.000000000,88.000000000,91.000000000,9.000000000,22545,860,59995292.000000000,2230706.000000000,1950356.000000000,28410,195637,144350,189526,178538,863420,223755,104716
3,200_500m,2545,0.000000000,24.000000000,45.000000000,74.000000000,83.000000000,86.000000000,14.000000000,3787,301,8598593.000000000,413218.000000000,111964.000000000,16330,16745,3940,6350,8941,25888,6601,13032
1,100m,177,0.000000000,12.000000000,43.000000000,69.000000000,76.000000000,86.000000000,14.000000000,411,19,1707668.000000000,35593.000000000,20804.000000000,1475,4132,3646,2101,1909,6020,361,956
0,100_200m,432,0.000000000,20.000000000,39.000000000,64.000000000,79.000000000,85.000000000,15.000000000,1082,63,2074931.000000000,108780.000000000,69048.000000000,11592,9854,3765,400,3218,3357,529,14657


In [18]:
######### SYNTHESE PAR EPCI et par ANNEE DES PERMIS responsables de l'artificialisation constatée entre 2014 et 2024 ##########

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt.groupby(["SIREN_EPCI","NOM_EPCI","ANNEE"], as_index=False).agg(Nb_Permis=("SIREN_EPCI","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"), 
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum"))

SITADEL_Bzh_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Bzh_Litt_Bilan["Permis_annules"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt_Bilan[['SIREN_EPCI','NOM_EPCI','ANNEE', 'Nb_Permis', 'Part_Permis_annules','Part_Demandeurs_MEME_EPCI',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Bzh_Litt_Bilan.to_csv("SITADEL/SITADEL_BZH_2014_2024_LITT_NEW2014_Bilan_EPCI_ANNEE.csv")
SITADEL_Bzh_Litt_Bilan.sample(5)

,SIREN_EPCI,NOM_EPCI,ANNEE,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
534,244400586,CC du Sud Estuaire,2023,3,0.000000000,100.000000000,67.000000000,0.000000000,67.000000000,33.000000000,1,0,3597.000000000,92.000000000,918.000000000,0,0,0,403,0,0,515,0
567,244400644,CA de la Région Nazairienne et de l'Estuaire (...,2023,9,0.000000000,11.000000000,67.000000000,67.000000000,67.000000000,33.000000000,9,0,2926.000000000,846.000000000,0.000000000,0,0,0,0,0,0,0,0
491,243500725,CC Côte d'Émeraude,2014,41,0.000000000,37.000000000,66.000000000,73.000000000,88.000000000,12.000000000,41,1,22432.000000000,5239.000000000,0.000000000,0,0,0,0,0,0,0,0
131,200067346,CA Pornic Agglo Pays de Retz,2022,28,0.000000000,36.000000000,68.000000000,71.000000000,96.000000000,4.000000000,41,1,31999.000000000,3897.000000000,0.000000000,0,0,0,0,0,0,0,0
48,200043123,CC Auray Quiberon Terre Atlantique,2023,22,0.000000000,36.000000000,64.000000000,73.000000000,82.000000000,18.000000000,29,3,12186.000000000,2756.000000000,1376.000000000,0,0,0,1376,0,0,0,0


In [19]:
######### SYNTHESE PAR BANDE LITTORALE ET PAR ANNEE DES PERMIS responsables de l'artificialisation constatée entre 2014 et 2024 ##########

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt.groupby(["BANDE_LITT","ANNEE"], as_index=False).agg(Nb_Permis=("BANDE_LITT","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"), 
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum"))

SITADEL_Bzh_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Bzh_Litt_Bilan["Permis_annules"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt_Bilan[['BANDE_LITT','ANNEE', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Bzh_Litt_Bilan.to_csv("SITADEL/SITADEL_BZH_2014_2024_LITT_NEW2014_Bilan_BANDELITT_ANNEE.csv")
SITADEL_Bzh_Litt_Bilan.sample(5)

,BANDE_LITT,ANNEE,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
79,500m_1km,2021,419,0.000000000,76.000000000,87.000000000,90.000000000,10.000000000,513,28,626800.000000000,51598.000000000,13453.000000000,43,1830,1202,1414,879,5212,1634,1130
50,200_500m,2021,169,0.000000000,72.000000000,80.000000000,84.000000000,16.000000000,238,22,1065548.000000000,25065.000000000,23431.000000000,9675,970,0,0,1517,1205,0,449
80,500m_1km,2022,209,0.000000000,72.000000000,84.000000000,90.000000000,10.000000000,281,17,258552.000000000,30684.000000000,2669.000000000,48,1757,402,0,0,454,8,0
54,200_500m,2013,103,0.000000000,78.000000000,89.000000000,89.000000000,11.000000000,154,11,109027.000000000,17515.000000000,5321.000000000,90,2582,257,698,0,0,837,857
43,1_5km,2019,1649,0.000000000,84.000000000,88.000000000,90.000000000,10.000000000,2077,63,3356928.000000000,213849.000000000,90127.000000000,960,8191,2618,11878,3865,38971,13451,7752


### Synthèse des permis localisés en dehors des secteurs déjà artificialisés en 2024 (Permis délivrés depuis janvier 2020)


In [20]:
SITADEL_Bzh_New2024=pd.read_csv("SITADEL/SITADEL_BZH_2020_2024_LITT_NEW2024.csv")

# Préparer les variables pour les synthèses :
SITADEL_Bzh_New2024.replace({"PERMIS_ANNULE": {"non": "0", "oui": "1"}}, inplace=True)
SITADEL_Bzh_New2024["PERMIS_ANNULE"]=SITADEL_Bzh_New2024["PERMIS_ANNULE"].astype("int64")
SITADEL_Bzh_New2024.replace({"RESID_SECOND": {"0.0": "0", "":"0", "1.0": "1"}}, inplace=True)
SITADEL_Bzh_New2024["RESID_SECOND"]=SITADEL_Bzh_New2024["RESID_SECOND"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SUPERF_TERRAIN"]=SITADEL_Bzh_New2024["SUPERF_TERRAIN"].astype("float64")
SITADEL_Bzh_New2024["SURF_HABIT_NEW"]=SITADEL_Bzh_New2024["SURF_HABIT_NEW"].astype("float64")
SITADEL_Bzh_New2024["SURF_LOCAUX_NEW"]=SITADEL_Bzh_New2024["SURF_LOCAUX_NEW"].astype("float64")
SITADEL_Bzh_New2024["SURF_HOTEL_NEW"]=SITADEL_Bzh_New2024["SURF_HOTEL_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_BUREAU_NEW"]=SITADEL_Bzh_New2024["SURF_BUREAU_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_COMMERC_NEW"]=SITADEL_Bzh_New2024["SURF_COMMERC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_ARTISAN_NEW"]=SITADEL_Bzh_New2024["SURF_ARTISAN_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_INDUS_NEW"]=SITADEL_Bzh_New2024["SURF_INDUS_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_AGRIC_NEW"]=SITADEL_Bzh_New2024["SURF_AGRIC_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_ENTREPOT_NEW"]=SITADEL_Bzh_New2024["SURF_ENTREPOT_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["SURF_SERV_PUB_NEW"]=SITADEL_Bzh_New2024["SURF_SERV_PUB_NEW"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_MEME_COMMUNE"]=SITADEL_Bzh_New2024["DEM_MEME_COMMUNE"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_MEME_EPCI"]=SITADEL_Bzh_New2024["DEM_MEME_EPCI"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_MEME_DEP"]=SITADEL_Bzh_New2024["DEM_MEME_DEP"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_BRETAGNE"]=SITADEL_Bzh_New2024["DEM_BRETAGNE"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_Ouest"]=SITADEL_Bzh_New2024["DEM_Ouest"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["DEM_LOINTAINS"]=SITADEL_Bzh_New2024["DEM_LOINTAINS"].fillna("0").astype("int64")
SITADEL_Bzh_New2024["NB_LOGEMENTS_NEW"]=SITADEL_Bzh_New2024["NB_LOGEMENTS_NEW"].fillna("0").astype("int64")

SITADEL_Bzh_New2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7404 entries, 0 to 7403
Data columns (total 59 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 7404 non-null   int64  
 1   ID_PERMIS                  7404 non-null   int64  
 2   Type_AU                    7404 non-null   object 
 3   DEP                        7404 non-null   int64  
 4   INSEE_COM                  7404 non-null   int64  
 5   NOM_COMMUNE                7404 non-null   object 
 6   SIREN_EPCI                 7404 non-null   object 
 7   NOM_EPCI                   7400 non-null   object 
 8   ID_LITT                    7404 non-null   object 
 9   BANDE_LITT                 7404 non-null   object 
 10  TacheUrb2014               7404 non-null   object 
 11  Hors_TacheUrb2014          7404 non-null   int64  
 12  Hors_TacheUrb2024          7404 non-null   int64  
 13  ZoneUrba                   5812 non-null   objec

In [21]:
######### SYNTHESE PAR COMMUNES DES PERMIS qui annoncent l'artificialisation 

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_New2024.groupby(["INSEE_COM","NOM_COMMUNE"], as_index=False).agg(Nb_Permis=("INSEE_COM","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"),
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum"))

SITADEL_Bzh_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Bzh_Litt_Bilan["Permis_annules"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_COMMUN"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_COMMUNE"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt_Bilan[['INSEE_COM','NOM_COMMUNE', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_COMMUN', 'Part_Demandeurs_MEME_EPCI',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Bzh_Litt_Bilan.to_csv("SITADEL/SITADEL_BZH_2020_2024_LITT_NEW2024_Bilan_COM.csv")
SITADEL_Bzh_Litt_Bilan.sample(5)

,INSEE_COM,NOM_COMMUNE,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_COMMUN,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
23,22176,Plédran,1,0.000000000,100.000000000,100.000000000,100.000000000,100.000000000,100.000000000,0.000000000,1,0,783.000000000,140.000000000,0.000000000,0,0,0,0,0,0,0,0
193,29251,Saint-Jean-du-Doigt,8,0.000000000,25.000000000,38.000000000,62.000000000,75.000000000,75.000000000,25.000000000,7,1,1988.000000000,697.000000000,1305.000000000,0,0,0,0,0,1305,0,0
91,29035,Coat-Méal,2,0.000000000,0.000000000,0.000000000,50.000000000,50.000000000,50.000000000,50.000000000,2,0,3008.000000000,236.000000000,0.000000000,0,0,0,0,0,0,0,0
142,29155,Ouessant,4,0.000000000,25.000000000,0.000000000,50.000000000,50.000000000,50.000000000,50.000000000,1,1,38464.000000000,94.000000000,335.000000000,0,0,189,0,0,0,50,96
320,56143,Muzillac,67,0.000000000,34.000000000,46.000000000,76.000000000,88.000000000,91.000000000,9.000000000,87,2,549983.000000000,8779.000000000,1148.000000000,0,254,0,0,0,515,0,0


In [22]:
######### SYNTHESE PAR EPCI DES PERMIS qui annoncent l'artificialisation 

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_New2024.groupby(["SIREN_EPCI","NOM_EPCI"], as_index=False).agg(Nb_Permis=("SIREN_EPCI","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"),
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum"))

SITADEL_Bzh_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Bzh_Litt_Bilan["Permis_annules"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt_Bilan[['SIREN_EPCI','NOM_EPCI', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_EPCI',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Bzh_Litt_Bilan.to_csv("SITADEL/SITADEL_BZH_2020_2024_LITT_NEW2024_Bilan_EPCI.csv")
SITADEL_Bzh_Litt_Bilan.sample(5)


,SIREN_EPCI,NOM_EPCI,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
5,200067072,CC Haut-Léon Communauté,205,0.000000000,60.000000000,80.000000000,83.000000000,84.000000000,16.000000000,209,29,1077623.000000000,23900.000000000,204629.000000000,0,475,856,3712,0,186774,1301,11491
0,200027027,CC Arc Sud Bretagne,113,0.000000000,44.000000000,71.000000000,85.000000000,88.000000000,12.000000000,126,9,1250800.000000000,12857.000000000,22868.000000000,0,348,474,292,0,20377,675,116
26,242900751,CC du Pays de Landivisiau,2,0.000000000,100.000000000,100.000000000,100.000000000,100.000000000,0.000000000,0,0,12600.000000000,0.000000000,3167.000000000,0,0,0,0,0,3167,0,0
32,243500782,CA du Pays de Saint Malo Agglomération,427,0.000000000,53.000000000,89.000000000,94.000000000,95.000000000,5.000000000,530,24,2522154.000000000,50240.000000000,100920.000000000,0,7894,1732,27217,17557,20558,24207,1637
6,200067247,CC Pleyben-Châteaulin-Porzay,31,0.000000000,55.000000000,84.000000000,87.000000000,90.000000000,10.000000000,37,1,569158.000000000,3332.000000000,2606.000000000,0,42,251,0,8,1955,99,0


In [23]:
######### SYNTHESE PAR BANDES LITTORALES DES PERMIS qui annoncent l'artificialisation 

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_New2024.groupby(["BANDE_LITT"], as_index=False).agg(Nb_Permis=("BANDE_LITT","count"),
   Permis_annules=("PERMIS_ANNULE","sum"),
   Nouveaux_Logements=("NB_LOGEMENTS_NEW","sum"),
   Resid_secondaire=("RESID_SECOND","sum"),
   Surface_terrains=("SUPERF_TERRAIN","sum"),
   Surface_Logements=("SURF_HABIT_NEW","sum"),
   Surface_Locaux=("SURF_LOCAUX_NEW","sum"),
   Surface_hotels=("SURF_HOTEL_NEW","sum"),
   Surface_bureaux=("SURF_BUREAU_NEW","sum"),
   Surface_commerces=("SURF_COMMERC_NEW","sum"),
   Surface_artisanat=("SURF_ARTISAN_NEW","sum"),
   Surface_industrie=("SURF_INDUS_NEW","sum"),
   Surface_bat_agricoles=("SURF_AGRIC_NEW","sum"),
   Surface_entrepots=("SURF_ENTREPOT_NEW","sum"),
   Surface_services_publics=("SURF_SERV_PUB_NEW","sum"),
   Demandeurs_MEME_COMMUNE=("DEM_MEME_COMMUNE","sum"),
   Demandeurs_MEME_EPCI=("DEM_MEME_EPCI","sum"),
   Demandeurs_MEME_DEP=("DEM_MEME_DEP","sum"),
   Demandeurs_BRETAGNE=("DEM_BRETAGNE","sum"),
   Demandeurs_Ouest=("DEM_Ouest","sum"),
   Demandeurs_LOINTAINS=("DEM_LOINTAINS","sum"))

SITADEL_Bzh_Litt_Bilan["Part_Permis_annules"]=((SITADEL_Bzh_Litt_Bilan["Permis_annules"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_EPCI"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_EPCI"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_MEME_DEP"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_MEME_DEP"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_BRETAGNE"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_BRETAGNE"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_Ouest"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_Ouest"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()
SITADEL_Bzh_Litt_Bilan["Part_Demandeurs_LOINTAINS"]=((SITADEL_Bzh_Litt_Bilan["Demandeurs_LOINTAINS"]/SITADEL_Bzh_Litt_Bilan["Nb_Permis"])*100).round()

SITADEL_Bzh_Litt_Bilan=SITADEL_Bzh_Litt_Bilan[['BANDE_LITT', 'Nb_Permis', 'Part_Permis_annules',
       'Part_Demandeurs_MEME_EPCI',
       'Part_Demandeurs_MEME_DEP', 'Part_Demandeurs_BRETAGNE',
       'Part_Demandeurs_Ouest', 'Part_Demandeurs_LOINTAINS','Nouveaux_Logements',
       'Resid_secondaire', 'Surface_terrains', 'Surface_Logements',
       'Surface_Locaux', 'Surface_hotels', 'Surface_bureaux',
       'Surface_commerces', 'Surface_artisanat', 'Surface_industrie',
       'Surface_bat_agricoles', 'Surface_entrepots',
       'Surface_services_publics']]

SITADEL_Bzh_Litt_Bilan.to_csv("SITADEL/SITADEL_BZH_2020_2024_LITT_NEW2024_Bilan_BANDELITT.csv")
SITADEL_Bzh_Litt_Bilan.sample(5)


,BANDE_LITT,Nb_Permis,Part_Permis_annules,Part_Demandeurs_MEME_EPCI,Part_Demandeurs_MEME_DEP,Part_Demandeurs_BRETAGNE,Part_Demandeurs_Ouest,Part_Demandeurs_LOINTAINS,Nouveaux_Logements,Resid_secondaire,Surface_terrains,Surface_Logements,Surface_Locaux,Surface_hotels,Surface_bureaux,Surface_commerces,Surface_artisanat,Surface_industrie,Surface_bat_agricoles,Surface_entrepots,Surface_services_publics
3,200_500m,666,0.000000000,46.000000000,73.000000000,81.000000000,84.000000000,16.000000000,1240,94,5341791.000000000,125116.000000000,60859.000000000,15791,8541,703,1697,7612,6975,1983,7277
1,100m,102,0.000000000,31.000000000,65.000000000,67.000000000,81.000000000,19.000000000,199,14,1556344.000000000,16683.000000000,14635.000000000,1475,3287,1926,1171,0,5775,184,712
4,500m_1km,1571,0.000000000,46.000000000,73.000000000,85.000000000,88.000000000,12.000000000,2170,166,7186791.000000000,223549.000000000,99742.000000000,6500,6851,7711,11689,3617,27434,19377,11694
0,100_200m,137,0.000000000,36.000000000,60.000000000,81.000000000,90.000000000,10.000000000,133,19,1480325.000000000,15786.000000000,12508.000000000,0,156,2096,0,1418,1758,529,5843
2,1_5km,4928,0.000000000,49.000000000,79.000000000,85.000000000,89.000000000,11.000000000,6827,255,33592653.000000000,674367.000000000,900391.000000000,12668,79925,60417,88073,88662,405426,109948,47349


## Les permis de 2024
(Permis délivrés depuis janvier 2024 en dehors des zones déjà artificialisées en 2024)

In [54]:
SITADEL_New2024=pd.read_csv("SITADEL/SITADEL_BZH_2020_2024_LITT_NEW2024.csv")
SITADEL_New2024=SITADEL_New2024[SITADEL_New2024["ANNEE"]==2024]

SITADEL_New2024.sort_values(by="INSEE_COM", inplace=True)
SITADEL_New2024.to_csv("SITADEL_BZH_2024_LITT_NEW.csv")
SITADEL_New2024.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1174 entries, 7302 to 5774
Data columns (total 59 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 1174 non-null   int64  
 1   ID_PERMIS                  1174 non-null   int64  
 2   Type_AU                    1174 non-null   object 
 3   DEP                        1174 non-null   int64  
 4   INSEE_COM                  1174 non-null   int64  
 5   NOM_COMMUNE                1174 non-null   object 
 6   SIREN_EPCI                 1174 non-null   object 
 7   NOM_EPCI                   1173 non-null   object 
 8   ID_LITT                    1174 non-null   object 
 9   BANDE_LITT                 1174 non-null   object 
 10  TacheUrb2014               1174 non-null   object 
 11  Hors_TacheUrb2014          1174 non-null   int64  
 12  Hors_TacheUrb2024          1174 non-null   int64  
 13  ZoneUrba                   915 non-null    object 